In [6]:
# ============================================================================
# META-ANALYSIS: Foreign Accent Reduction EFFECTIVENESS
# ============================================================================
# PURPOSE: Quantify training effectiveness and isolate moderators via 
#          random-effects models, robustness checks, and bias assessment
#
# WORKFLOW:
#   STEP 0: Environment setup & data validation
#   STEP 1: Overall random-effects model (REML, Q/I²/τ², prediction interval)
#   STEP 2: Moderator analyses (univariate & multivariate meta-regression)
#   STEP 3: Robustness checks (leave-one-out, influence, RVE)
#   STEP 4: Publication bias (Egger, trim-and-fill, fail-safe N)
#
# DEPENDENCIES:
#   metafor  ≥ 3.0.0    REML meta-analysis (Viechtbauer, 2010)
#   robumeta ≥ 2.0      Robust variance estimation (Hedges et al., 2010)
#
# ============================================================================
# DATA STRUCTURE
# ============================================================================
#
# INPUT:  Meta_ready_cleaned.csv
#
# OUTPUTS (14 files total, organized by analysis step)
# 
# Meta_Analysis_Results/
# ├── Step1_Overall_Model/
# │   ├── overall_meta_analysis_results.csv       Pooled effect & heterogeneity
# │   └── forest_overall.png                      Forest plot for overall effect
# ├── Step2_Moderator_Analysis/
# │   ├── univariate_moderator_summary.csv        Omnibus tests per moderator (1 row/mod)
# │   ├── univariate_moderator_results.csv        Level-by-level results (β, CI, p)
# │   ├── multivariate_model_coefficients.csv     Adjusted moderator effects
# │   └── multivariate_model_fit.csv              Model R² & fit indices
# ├── Step3_Robustness_Checks/
# │   ├── leave_one_out_analysis.csv              Sensitivity metrics
# │   ├── influence_diagnostics.csv               Cook's D, DFBETAS, leverage
# │   ├── rve_overall_effect.csv                  RVE-adjusted pooled effect
# │   └── rve_moderator_results.csv               RVE-adjusted moderator tests
# └── Step4_Publication_Bias/
#     ├── funnel_plot.png                         Visual asymmetry assessment
#     ├── publication_bias_tests.csv              Egger, trim-and-fill, fsN
#     └── publication_bias_trim_and_fill.csv      Imputed studies (only if k_imputed > 0)
#
# ============================================================================

# Clear workspace & initialize
rm(list = ls())
analysis_start_time <- Sys.time()

cat("\n", paste0(rep("=", 80), collapse = ""), "\n", sep = "")
cat("Foreign Accent Reduction META-ANALYSIS\n")
cat(paste0(rep("=", 80), collapse = ""), "\n")
cat("Initiated: ", format(analysis_start_time, "%Y-%m-%d %H:%M:%S"), " | ",
    R.version.string, "\n", sep = "")
cat(paste0(rep("=", 80), collapse = ""), "\n\n", sep = "")

# ============================================================================
# STEP 0: ENVIRONMENT SETUP & DATA VALIDATION
# ============================================================================

cat(paste0(rep("=", 80), collapse = ""), "\n")
cat("STEP 0: ENVIRONMENT SETUP & DATA VALIDATION\n")
cat(paste0(rep("=", 80), collapse = ""), "\n\n")

# ----------------------------------------------------------------------------
# 0.1: Package Installation & Loading
# ----------------------------------------------------------------------------
cat("0.1: Loading required packages...\n", paste0(rep("-", 80), collapse = ""), "\n", sep = "")

required_packages <- c("metafor", "robumeta")
for (pkg in required_packages) {
  if (!require(pkg, character.only = TRUE, quietly = TRUE)) {
    install.packages(pkg, repos = "https://cloud.r-project.org/", quiet = TRUE)
  }
  library(pkg, character.only = TRUE)
  cat("  ✅ ", pkg, " v", as.character(packageVersion(pkg)), "\n", sep = "")
}
cat("\n")

# ----------------------------------------------------------------------------
# 0.2: Create Output Directory Structure
# ----------------------------------------------------------------------------
cat("0.2: Setting up output directories...\n", paste0(rep("-", 80), collapse = ""), "\n", sep = "")

# Define main results directory
results_dir <- file.path(getwd(), "Meta_Analysis_Results")

# Create subdirectories for each analysis step
step1_dir <- file.path(results_dir, "Step1_Overall_Model")
step2_dir <- file.path(results_dir, "Step2_Moderator_Analysis")
step3_dir <- file.path(results_dir, "Step3_Robustness_Checks")
step4_dir <- file.path(results_dir, "Step4_Publication_Bias")

# Create all directories
for (dir_path in c(step1_dir, step2_dir, step3_dir, step4_dir)) {
  if (!dir.exists(dir_path)) {
    dir.create(dir_path, recursive = TRUE)
  }
}

cat("  ✅ Created output structure:\n")
cat("     - Meta_Analysis_Results/\n")
cat("       ├── Step1_Overall_Model/\n")
cat("       ├── Step2_Moderator_Analysis/\n")
cat("       ├── Step3_Robustness_Checks/\n")
cat("       └── Step4_Publication_Bias/\n\n")

# Store paths in global environment for use by helper functions
assign("step1_dir", step1_dir, envir = .GlobalEnv)
assign("step2_dir", step2_dir, envir = .GlobalEnv)
assign("step3_dir", step3_dir, envir = .GlobalEnv)
assign("step4_dir", step4_dir, envir = .GlobalEnv)

# ----------------------------------------------------------------------------
# 0.3: Define Helper Function (Safe CSV Export with Error Handling)
# ----------------------------------------------------------------------------
cat("0.3: Defining helper functions...\n", paste0(rep("-", 80), collapse = ""), "\n", sep = "")

safe_write_csv <- function(data, filename, output_dir = NULL, show_message = TRUE) {
  # Use provided directory or fall back to working directory
  if (is.null(output_dir)) {
    if (exists("output_path", envir = .GlobalEnv)) {
      output_dir <- get("output_path", envir = .GlobalEnv)
    } else {
      output_dir <- getwd()
    }
  }
  
  tryCatch({
    write.csv(data, file.path(output_dir, filename), row.names = FALSE, fileEncoding = "UTF-8")
    if (show_message) cat("  ✅ Saved: ", filename, "\n", sep = "")
    return(TRUE)
  }, error = function(e) {
    timestamp <- format(Sys.time(), "%Y%m%d_%H%M%S")
    alt_filename <- paste0(sub("\\.csv$", "", filename), "_", timestamp, ".csv")
    write.csv(data, file.path(output_dir, alt_filename), row.names = FALSE, fileEncoding = "UTF-8")
    if (show_message) cat("  ⚠️  File locked. Saved as: ", alt_filename, "\n", sep = "")
    return(FALSE)
  })
}

cat("  ✅ Safe CSV writer configured\n\n")

# ----------------------------------------------------------------------------
# 0.4: Configure R Environment
# ----------------------------------------------------------------------------
cat("0.4: Configuring R environment...\n", paste0(rep("-", 80), collapse = ""), "\n", sep = "")

options(digits = 4, scipen = 999, width = 80, stringsAsFactors = FALSE, warn = 1)
cat("  ✅ Precision: 4 decimals | No scientific notation | 80-char width\n\n")

# ----------------------------------------------------------------------------
# 0.5: Load & Validate Dataset
# ----------------------------------------------------------------------------
cat("0.5: Loading meta-analysis dataset...\n", paste0(rep("-", 80), collapse = ""), "\n", sep = "")

data_file <- "Meta_ready_cleaned.csv"
if (!file.exists(data_file)) {
  stop("Data file missing: ", data_file, " | Check working directory: ", getwd())
}

df <- read.csv(data_file, stringsAsFactors = FALSE, fileEncoding = "UTF-8")
cat("  ✅ Loaded: ", data_file, " | Effects: ", nrow(df), " | Studies: ", 
    length(unique(df$Study_ID)), " | Columns: ", ncol(df), "\n\n", sep = "")

# ----------------------------------------------------------------------------
# 0.6: Validate Essential Columns
# ----------------------------------------------------------------------------
cat("0.6: Validating dataset structure...\n", paste0(rep("-", 80), collapse = ""), "\n", sep = "")

essential_cols <- c("Study_ID", "Effect_ID", "Hedges_g")
missing_cols <- setdiff(essential_cols, names(df))
if (length(missing_cols) > 0) stop("Missing columns: ", paste(missing_cols, collapse = ", "))

if ("SE" %in% names(df)) {
  variance_source <- "SE"
  df$vi <- df$SE^2  # Create variance from standard error
  cat("  ✅ Variance source: Standard Error (SE)\n")
  cat("  ✅ Variance (vi) calculated from SE: vi = SE²\n")
} else if ("Variance" %in% names(df)) {
  variance_source <- "Variance"
  df$vi <- df$Variance  # Use existing variance
  cat("  ✅ Variance source: Variance\n")
  cat("  ✅ Variance (vi) assigned from Variance column\n")
} else {
  stop("Missing variance metric: Dataset must include 'SE' or 'Variance' column")
}

cat("  ✅ Dataset validated successfully\n\n")

# ----------------------------------------------------------------------------
# 0.7: Set Reference Levels for Categorical Moderators
# ----------------------------------------------------------------------------
cat("0.7: Setting reference levels for categorical moderators...\n", paste0(rep("-", 80), collapse = ""), "\n", sep = "")

# Initialize baseline storage list
BASELINE_LEVELS <- list()

# LEARNER CHARACTERISTICS
# Adolescent as reference (youngest age group baseline)
if ("Age_Group" %in% names(df)) {
  BASELINE_Age_Group <- "Adolescent"
  BASELINE_LEVELS$Age_Group <- BASELINE_Age_Group
  df$Age_Group <- relevel(factor(df$Age_Group), ref = BASELINE_Age_Group)
  cat("  ✅ Age_Group: reference = ", BASELINE_Age_Group, "\n", sep = "")
}

# Advanced as reference (highest proficiency baseline)
if ("Proficiency_Level" %in% names(df)) {
  BASELINE_Proficiency_Level <- "Beginner"
  BASELINE_LEVELS$Proficiency_Level <- BASELINE_Proficiency_Level
  df$Proficiency_Level <- relevel(factor(df$Proficiency_Level), ref = BASELINE_Proficiency_Level)
  cat("  ✅ Proficiency_Level: reference = ", BASELINE_Proficiency_Level, "\n", sep = "")
}

# Non-English major as reference (typical baseline)
if ("English_Major" %in% names(df)) {
  BASELINE_English_Major <- "No"
  BASELINE_LEVELS$English_Major <- BASELINE_English_Major
  df$English_Major <- relevel(factor(df$English_Major), ref = BASELINE_English_Major)
  cat("  ✅ English_Major: reference = ", BASELINE_English_Major, "\n", sep = "")
}

# Secondary school as reference (lower education level baseline)
if ("Education_Stage" %in% names(df)) {
  BASELINE_Education_Stage <- "SecondarySchool"
  BASELINE_LEVELS$Education_Stage <- BASELINE_Education_Stage
  df$Education_Stage <- relevel(factor(df$Education_Stage), ref = BASELINE_Education_Stage)
  cat("  ✅ Education_Stage: reference = ", BASELINE_Education_Stage, "\n", sep = "")
}

# LEARNING ENVIRONMENT
# Foreign language context as reference (more common than L2)
if ("Learning_Context" %in% names(df)) {
  BASELINE_Learning_Context <- "English"
  BASELINE_LEVELS$Learning_Context <- BASELINE_Learning_Context
  df$Learning_Context <- relevel(factor(df$Learning_Context), ref = BASELINE_Learning_Context)
  cat("  ✅ Learning_Context: reference = ", BASELINE_Learning_Context, "\n", sep = "")
}

# Laboratory as reference (controlled setting baseline)
if ("Training_Context" %in% names(df)) {
  BASELINE_Training_Context <- "Laboratory"
  BASELINE_LEVELS$Training_Context <- BASELINE_Training_Context
  df$Training_Context <- relevel(factor(df$Training_Context), ref = BASELINE_Training_Context)
  cat("  ✅ Training_Context: reference = ", BASELINE_Training_Context, "\n", sep = "")
}

# INSTRUCTIONAL FEATURES
# Perception as reference (receptive skill baseline)
if ("Focus_Type" %in% names(df)) {
  BASELINE_Focus_Type <- "Production"
  BASELINE_LEVELS$Focus_Type <- BASELINE_Focus_Type
  df$Focus_Type <- relevel(factor(df$Focus_Type), ref = BASELINE_Focus_Type)
  cat("  ✅ Focus_Type: reference = ", BASELINE_Focus_Type, "\n", sep = "")
}

# Segmental as reference (traditional pronunciation target)
if ("Target_Feature" %in% names(df)) {
  BASELINE_Target_Feature <- "Suprasegmental"
  BASELINE_LEVELS$Target_Feature <- BASELINE_Target_Feature
  df$Target_Feature <- relevel(factor(df$Target_Feature), ref = BASELINE_Target_Feature)
  cat("  ✅ Target_Feature: reference = ", BASELINE_Target_Feature, "\n", sep = "")
}

# Implicit feedback as reference (less explicit baseline)
if ("Feedback_Type" %in% names(df)) {
  BASELINE_Feedback_Type <- "Implicit"
  BASELINE_LEVELS$Feedback_Type <- BASELINE_Feedback_Type
  df$Feedback_Type <- relevel(factor(df$Feedback_Type), ref = BASELINE_Feedback_Type)
  cat("  ✅ Feedback_Type: reference = ", BASELINE_Feedback_Type, "\n", sep = "")
}

# Human instructor as reference (traditional instruction mode)
if ("Instructor_Type" %in% names(df)) {
  BASELINE_Instructor_Type <- "Human"
  BASELINE_LEVELS$Instructor_Type <- BASELINE_Instructor_Type
  df$Instructor_Type <- relevel(factor(df$Instructor_Type), ref = BASELINE_Instructor_Type)
  cat("  ✅ Instructor_Type: reference = ", BASELINE_Instructor_Type, "\n", sep = "")
}

# No peer interaction as reference (typical baseline)
if ("Peer_Interaction" %in% names(df)) {
  BASELINE_Peer_Interaction <- "No"
  BASELINE_LEVELS$Peer_Interaction <- BASELINE_Peer_Interaction
  df$Peer_Interaction <- relevel(factor(df$Peer_Interaction), ref = BASELINE_Peer_Interaction)
  cat("  ✅ Peer_Interaction: reference = ", BASELINE_Peer_Interaction, "\n", sep = "")
}

# No visual cue as reference (audio-only baseline)
if ("Visual_Cue" %in% names(df)) {
  BASELINE_Visual_Cue <- "No"
  BASELINE_LEVELS$Visual_Cue <- BASELINE_Visual_Cue
  df$Visual_Cue <- relevel(factor(df$Visual_Cue), ref = BASELINE_Visual_Cue)
  cat("  ✅ Visual_Cue: reference = ", BASELINE_Visual_Cue, "\n", sep = "")
}

# Long duration as reference (extended treatment baseline)
if ("Treatment_Duration" %in% names(df)) {
  BASELINE_Treatment_Duration <- "Long"
  BASELINE_LEVELS$Treatment_Duration <- BASELINE_Treatment_Duration
  df$Treatment_Duration <- relevel(factor(df$Treatment_Duration), ref = BASELINE_Treatment_Duration)
  cat("  ✅ Treatment_Duration: reference = ", BASELINE_Treatment_Duration, "\n", sep = "")
}

# METHODOLOGICAL FEATURES
# Passive comparator as reference (no-treatment baseline)
if ("Comparator_Type" %in% names(df)) {
  BASELINE_Comparator_Type <- "Passive"
  BASELINE_LEVELS$Comparator_Type <- BASELINE_Comparator_Type
  df$Comparator_Type <- relevel(factor(df$Comparator_Type), ref = BASELINE_Comparator_Type)
  cat("  ✅ Comparator_Type: reference = ", BASELINE_Comparator_Type, "\n", sep = "")
}

# Quasi-experimental as reference (most common design)
if ("Design_Type" %in% names(df)) {
  BASELINE_Design_Type <- "Quasi_Experiment"
  BASELINE_LEVELS$Design_Type <- BASELINE_Design_Type
  df$Design_Type <- relevel(factor(df$Design_Type), ref = BASELINE_Design_Type)
  cat("  ✅ Design_Type: reference = ", BASELINE_Design_Type, "\n", sep = "")
}

# Pronunciation accuracy as reference (most common outcome)
if ("Outcome_Domain" %in% names(df)) {
  BASELINE_Outcome_Domain <- "Pronunciation_Accuracy"
  BASELINE_LEVELS$Outcome_Domain <- BASELINE_Outcome_Domain
  df$Outcome_Domain <- relevel(factor(df$Outcome_Domain), ref = BASELINE_Outcome_Domain)
  cat("  ✅ Outcome_Domain: reference = ", BASELINE_Outcome_Domain, "\n", sep = "")
}

# Human rater as reference (traditional assessment)
if ("Rater_Type" %in% names(df)) {
  BASELINE_Rater_Type <- "Human"
  BASELINE_LEVELS$Rater_Type <- BASELINE_Rater_Type
  df$Rater_Type <- relevel(factor(df$Rater_Type), ref = BASELINE_Rater_Type)
  cat("  ✅ Rater_Type: reference = ", BASELINE_Rater_Type, "\n", sep = "")
}

# L1 - set most common or theoretically meaningful L1 as reference
# For this dataset, we can check which L1 has the most observations
if ("L1" %in% names(df)) {
  l1_counts <- table(df$L1)
  BASELINE_L1 <- "Arabic"
  BASELINE_LEVELS$L1 <- BASELINE_L1
  df$L1 <- relevel(factor(df$L1), ref = BASELINE_L1)
  cat("  ✅ L1: reference = ", BASELINE_L1, " (most common, auto-detected)\n", sep = "")
}

cat("\n  ✅ Reference levels set for all categorical moderators\n")
cat("  ✅ Baseline levels stored in BASELINE_LEVELS list\n\n")

cat(paste0(rep("=", 80), collapse = ""), "\n\n")


Foreign Accent Reduction META-ANALYSIS
Initiated: 2025-11-30 14:48:37 | R version 4.5.1 (2025-06-13 ucrt)

STEP 0: ENVIRONMENT SETUP & DATA VALIDATION

0.1: Loading required packages...
--------------------------------------------------------------------------------
  ✅ metafor v4.8.0
  ✅ robumeta v2.1

0.2: Setting up output directories...
--------------------------------------------------------------------------------
  ✅ Created output structure:
     - Meta_Analysis_Results/
       ├── Step1_Overall_Model/
       ├── Step2_Moderator_Analysis/
       ├── Step3_Robustness_Checks/
       └── Step4_Publication_Bias/

0.3: Defining helper functions...
--------------------------------------------------------------------------------
  ✅ Safe CSV writer configured

0.4: Configuring R environment...
--------------------------------------------------------------------------------
  ✅ Precision: 4 decimals | No scientific notation | 80-char width

0.5: Loading meta-analysis dataset...
------

In [7]:
# ============================================================================
# STEP 1: OVERALL RANDOM-EFFECTS META-ANALYSIS
# ============================================================================
# PURPOSE: Estimate pooled training effectiveness & assess heterogeneity
# MODEL:   yi ~ N(θ + ui, vi) where ui ~ N(0, τ²)
#          REML estimation | Z-test | 95% CI & prediction interval
#
# OUTPUTS (in Meta_Analysis_Results/Step1_Overall_Model/):
#   1. overall_meta_analysis_results.csv  - Pooled effect size & heterogeneity statistics
#   2. forest_overall.png                 - Forest plot visualization of all studies
# ============================================================================

cat(paste0(rep("=", 80), collapse = ""), "\n")
cat("STEP 1: OVERALL RANDOM-EFFECTS META-ANALYSIS\n")
cat(paste0(rep("=", 80), collapse = ""), "\n")
cat("Research Question: Is L2 pronunciation training effective overall?\n\n")

# ----------------------------------------------------------------------------
# 1.1: Fit REML Random-Effects Model
# ----------------------------------------------------------------------------
cat("1.1: Fitting random-effects model (REML)...\n", paste0(rep("-", 80), collapse = ""), "\n", sep = "")

# Explicitly use df$vi for clarity and robustness
res_overall <- rma(yi = Hedges_g, vi = df$vi, data = df, method = "REML", test = "z")
cat("  ✅ Model fitted | Method: REML | Test: Z-distribution\n\n")

# ----------------------------------------------------------------------------
# 1.2: Overall Effect Size
# ----------------------------------------------------------------------------
cat("1.2: Overall effect size estimate...\n", paste0(rep("-", 80), collapse = ""), "\n\n", sep = "")

overall_g <- res_overall$b[1]
overall_se <- res_overall$se
overall_ci_lb <- res_overall$ci.lb
overall_ci_ub <- res_overall$ci.ub
overall_z <- res_overall$zval
overall_p <- res_overall$pval

sig_marker <- if (overall_p < 0.001) " ***" else if (overall_p < 0.01) " **" else if (overall_p < 0.05) " *" else ""
sig_interpretation <- if (overall_p < 0.001) "HIGHLY SIGNIFICANT (p < .001)" else 
                      if (overall_p < 0.01) "Very significant (p < .01)" else 
                      if (overall_p < 0.05) "Significant (p < .05)" else 
                      "Not significant (p ≥ .05)"

cat("  OVERALL EFFECT (Hedges' g):\n  ", paste0(rep("-", 76), collapse = ""), "\n", sep = "")
cat("    g = ", sprintf("%.4f", overall_g), ", SE = ", sprintf("%.4f", overall_se), 
    ", 95% CI [", sprintf("%.4f", overall_ci_lb), ", ", sprintf("%.4f", overall_ci_ub), "]\n", sep = "")
cat("    Z = ", sprintf("%.4f", overall_z), ", p = ", sprintf("%.4f", overall_p), sig_marker, "\n", sep = "")
cat("    ", sig_interpretation, "\n\n", sep = "")

# ----------------------------------------------------------------------------
# 1.3: Heterogeneity Assessment
# ----------------------------------------------------------------------------
cat("1.3: Heterogeneity statistics...\n", paste0(rep("-", 80), collapse = ""), "\n\n", sep = "")

Q_stat <- res_overall$QE
Q_df <- res_overall$k - 1
Q_p <- res_overall$QEp
I2 <- res_overall$I2
tau2 <- res_overall$tau2
tau <- sqrt(tau2)
H2 <- res_overall$H2

cat("  HETEROGENEITY:\n  ", paste0(rep("-", 76), collapse = ""), "\n", sep = "")
# Improved Q statistic formatting
cat("    Q = ", sprintf("%.4f", Q_stat), " (df = ", Q_df, 
    if (Q_p < 0.001) ", p < .001 ***)\n" else paste0(", p = ", sprintf("%.4f", Q_p), ")\n"), sep = "")
cat("    I² = ", sprintf("%.2f", I2), "% | τ² = ", sprintf("%.4f", tau2), 
    " | H² = ", sprintf("%.4f", H2), "\n\n", sep = "")

I2_interpretation <- if (I2 < 25) "Low heterogeneity (I² < 25%)" else 
                     if (I2 < 50) "Moderate heterogeneity (25% ≤ I² < 50%)" else 
                     if (I2 < 75) "Substantial heterogeneity (50% ≤ I² < 75%)" else 
                     "Very high heterogeneity (I² ≥ 75%)"

cat("  I² Interpretation: ", I2_interpretation, "\n", sep = "")
# Use consistent threshold: I2 >= 50
if (I2 >= 50) cat("    → ✅ MODERATOR ANALYSIS WARRANTED\n")
cat("\n")

# Calculate 95% prediction interval
# PI = estimate ± t(k-2) × sqrt(τ² + SE²)
k <- res_overall$k
t_crit <- qt(0.975, df = k - 2)
pi_lower <- overall_g - t_crit * sqrt(tau2 + overall_se^2)
pi_upper <- overall_g + t_crit * sqrt(tau2 + overall_se^2)

cat("  95% PREDICTION INTERVAL:\n")
cat("  ", paste0(rep("-", 76), collapse = ""), "\n", sep = "")
cat("    Interval: [", sprintf("%.4f", pi_lower), ", ", 
    sprintf("%.4f", pi_upper), "]\n", sep = "")
cat("\n")
cat("  Interpretation:\n")
cat("    → In 95% of contexts, the true effect is expected to fall\n")
cat("      within this range\n")
cat("    → Wide interval indicates substantial heterogeneity\n")

cat("\n")

# ----------------------------------------------------------------------------
# Step 1.5: Summary and Interpretation
# ----------------------------------------------------------------------------
cat("Step 1.5: Overall meta-analysis summary...\n")
cat(paste0(rep("-", 80), collapse = ""), "\n\n")

cat("  SUMMARY:\n")
cat("  ", paste0(rep("=", 76), collapse = ""), "\n", sep = "")
cat("\n")

# Effect size interpretation
cat("  1. EFFECT SIZE:\n")
cat("     → Training effect: g = ", sprintf("%.4f", overall_g), 
    " [", sprintf("%.4f", overall_ci_lb), ", ", 
    sprintf("%.4f", overall_ci_ub), "]\n", sep = "")
cat("     → Interpretation:  ", sig_interpretation, "\n", sep = "")

# Effect size magnitude (Cohen's benchmarks)
if (abs(overall_g) < 0.2) {
  magnitude <- "Negligible to small"
} else if (abs(overall_g) < 0.5) {
  magnitude <- "Small to medium"
} else if (abs(overall_g) < 0.8) {
  magnitude <- "Medium to large"
} else {
  magnitude <- "Large to very large"
}
cat("     → Magnitude:       ", magnitude, " (Cohen, 1988)\n", sep = "")

cat("\n")

cat("  2. HETEROGENEITY:\n")
cat("     → ", I2_interpretation, "\n", sep = "")
# Use consistent threshold: I2 >= 50
if (I2 >= 50) {
  cat("     → CONCLUSION: Moderator analysis is justified\n")
  proceed_to_moderators <- TRUE
} else {
  cat("     → CONCLUSION: Moderator analysis optional\n")
  proceed_to_moderators <- FALSE
}

cat("\n")

cat("  3. NEXT STEPS:\n")
if (proceed_to_moderators) {
  cat("     ✅ Proceed to STEP 2: Moderator Analyses\n")
  cat("        (High I² indicates systematic variation to explain)\n")
} else {
  cat("     → Consider exploratory moderator analyses\n")
  cat("        (Low-moderate I² suggests limited systematic variation)\n")
}

cat("\n")

# ----------------------------------------------------------------------------
# Step 1.6: Export Overall Results
# ----------------------------------------------------------------------------
cat("Step 1.6: Exporting overall meta-analysis results...\n", paste0(rep("-", 80), collapse = ""), "\n", sep = "")

overall_results <- data.frame(
  Statistic = c("Effect Size (g)", "SE", "95% CI Lower", "95% CI Upper", 
                "Z-value", "p-value", "Q", "Q df", "Q p-value", 
                "I²", "τ²", "H²", "PI Lower", "PI Upper"),
  Value = c(overall_g, overall_se, overall_ci_lb, overall_ci_ub, 
            overall_z, overall_p, Q_stat, Q_df, Q_p, 
            I2, tau2, H2, pi_lower, pi_upper)
)

safe_write_csv(overall_results, "overall_meta_analysis_results.csv", step1_dir)

cat("\n")

# ---------------------[FOREST PLOT GENERATION]---------------------
# Step 1.7: Generate Forest Plot for Overall Effect
# ---------------------[New: Forest plot for overall meta-analysis]---------------------
cat("Step 1.7: Generating forest plot for overall effect...\n", paste0(rep("-", 80), collapse = ""), "\n", sep = "")

# Create forest plot
forest_filename <- file.path(step1_dir, "forest_overall.png")
png(forest_filename, width = 1400, height = 900, res = 120)

forest(res_overall,
       header = TRUE,
       xlim = c(-3, 5),
       at = c(-1, -0.5, 0, 0.5, 1, 1.5),
       xlab = "Hedges' g",
       slab = paste(df$Study_ID),
       main = paste0("Forest Plot: Overall Effect (g = ", sprintf("%.4f", overall_g), 
                     ", 95% CI [", sprintf("%.4f", overall_ci_lb), ", ", 
                     sprintf("%.4f", overall_ci_ub), "], p = ", sprintf("%.4f", overall_p), ")"))

dev.off()

cat("  ✅ Forest plot saved to: ", forest_filename, "\n", sep = "")
cat("\n")

cat("✅ STEP 1 COMPLETE: Overall random-effects meta-analysis finished\n")
cat(paste0(rep("=", 80), collapse = ""), "\n\n")


STEP 1: OVERALL RANDOM-EFFECTS META-ANALYSIS
Research Question: Is L2 pronunciation training effective overall?

1.1: Fitting random-effects model (REML)...
--------------------------------------------------------------------------------
  ✅ Model fitted | Method: REML | Test: Z-distribution

1.2: Overall effect size estimate...
--------------------------------------------------------------------------------

  OVERALL EFFECT (Hedges' g):
  ----------------------------------------------------------------------------
    g = 0.5272, SE = 0.0761, 95% CI [0.3781, 0.6763]
    Z = 6.9306, p = 0.0000 ***
    HIGHLY SIGNIFICANT (p < .001)

1.3: Heterogeneity statistics...
--------------------------------------------------------------------------------

  HETEROGENEITY:
  ----------------------------------------------------------------------------
    Q = 60.4647 (df = 28, p < .001 ***)
    I² = 54.64% | τ² = 0.0882 | H² = 2.2047

  I² Interpretation: Substantial heterogeneity (50% ≤ I² < 75%)

[1] TRUE


Step 1.7: Generating forest plot for overall effect...
--------------------------------------------------------------------------------


agg_record_1588552183 
                    2

  ✅ Forest plot saved to: c:/Users/Lenovo/Desktop/github/Meta_Analysis_Results/Step1_Overall_Model/forest_overall.png

✅ STEP 1 COMPLETE: Overall random-effects meta-analysis finished



In [8]:
# ============================================================================
# STEP 2: MODERATOR ANALYSES
# ============================================================================
#
# OUTPUT FILES GENERATED IN STEP 2 (in Meta_Analysis_Results/Step2_Moderator_Analysis/):
#
# (A) UNIVARIATE ANALYSES (STEP 2.1)
#     1. univariate_moderator_summary.csv
#        • Contains: One summary row per moderator with omnibus statistics
#        • Columns: Moderator, k, QM, df_QM, p_QM, tau2, R²
#        • Use for: Overview table, identifying significant moderators
#
#     2. univariate_moderator_results.csv (NEW)
#        • Contains: Level-by-level detailed results for each moderator
#        • Columns: Moderator, Level, Type, n, Estimate, SE, CI_Lower, CI_Upper,
#                   z_value, p_value, QM, QM_p, tau2_residual, I2_residual, R2
#        • Rows: One per moderator level (categorical) or per moderator (continuous)
#        • Use for: Forest plots, detailed visualization, level-specific inference
#
# (B) MULTIVARIATE ANALYSIS (STEP 2.2)
#     *Generated only if ≥ 1 significant moderator from STEP 2.1*
#
#     3. multivariate_model_coefficients.csv
#        • Regression coefficients for each moderator (adjusted model)
#        • Format: Moderator_Family + Level_Label
#        • Includes: β, SE, CI, z, p
#        • Categorical: Multiple rows per moderator (one per level + baseline)
#        • Continuous: One row (Family = Label = moderator name)
#
#     4. multivariate_model_fit.csv
#        • Overall model statistics
#        • Includes: QM, QE, τ², I², pseudo-R², k, p
#
# NOTE: Forest and funnel plots are NOT generated in Step 2
# ============================================================================

cat("###############################################################################\n")
cat("#                   STEP 2: MODERATOR META-ANALYSIS                          #\n")
cat("###############################################################################\n\n")

# Verify STEP 1 dependencies
cat("Verifying STEP 1 dependencies...\n")
cat(paste0(rep("-", 80), collapse = ""), "\n")

if (!exists("tau2")) {
  stop("ERROR: Global variable 'tau2' not found. Please run STEP 1 first.\n",
       "       STEP 2 requires tau2 for R² calculation: (tau2 - tau2_res) / tau2")
}

if (!exists("res_overall")) {
  stop("ERROR: Overall meta-analysis results not found. Please run STEP 1 first.\n",
       "       STEP 2 requires res_overall for model comparison.")
}

cat("  ✅ tau2 found: τ² = ", tau2, "\n", sep = "")
cat("  ✅ res_overall found: k = ", res_overall$k, "\n", sep = "")
cat("\n")

# ----------------------------------------------------------------------------
# Step 2.1: Univariate Moderator Analysis
# ----------------------------------------------------------------------------
cat("================================================================================\n")
cat("STEP 2.1: UNIVARIATE MODERATOR ANALYSIS\n")
cat("================================================================================\n\n")

# ----------------------------------------------------------------------------
# Step 2.1.1: Define Moderator Categories
# ----------------------------------------------------------------------------
cat("Step 2.1.1: Defining moderator variables...\n")
cat(paste0(rep("-", 80), collapse = ""), "\n\n")

# Category 1: Participant Characteristics
participant_moderators <- c(
  "L1",                    # Native language
  "Age_Group",             # Age group
  "Proficiency_Level",     # Proficiency level
  "English_Major",         # English major status
  "Education_Stage"        # Education level
)

# Category 2: Treatment Characteristics
treatment_moderators <- c(
  "Treatment_Duration",    # Duration category
  "Training_TotalMinute",  # Total minutes
  "Training_TotalWeeks",   # Total weeks
  "Training_Context",      # Training context
  "Feedback_Type",         # Feedback type
  "Visual_Cue",            # Visual cue usage
  "Instructor_Type"        # Instructor type
)

# Category 3: Linguistic Features
linguistic_moderators <- c(
  "Target_Feature",        # Target feature
  "Focus_Type"             # Focus type
)

# Category 4: Methodological Features
method_moderators <- c(
  "Design_Type",           # Study design
  "Comparator_Type",       # Comparator type
  "Learning_Context",      # Learning context
  "Rater_Type",            # Rater type
  "Peer_Interaction"       # Peer interaction
)

# Category 5: Outcome Characteristics
outcome_moderators <- c(
  "Outcome_Domain"         # Outcome domain
)

# Combine all moderators
all_moderators <- c(
  participant_moderators,
  treatment_moderators,
  linguistic_moderators,
  method_moderators,
  outcome_moderators
)

# Display moderator categories
cat("  Category 1: Participant Characteristics (n = ", length(participant_moderators), ")\n", sep = "")
for (i in seq_along(participant_moderators)) {
  cat("    ", i, ". ", participant_moderators[i], "\n", sep = "")
}
cat("\n")

cat("  Category 2: Treatment Characteristics (n = ", length(treatment_moderators), ")\n", sep = "")
for (i in seq_along(treatment_moderators)) {
  cat("    ", i, ". ", treatment_moderators[i], "\n", sep = "")
}
cat("\n")

cat("  Category 3: Linguistic Features (n = ", length(linguistic_moderators), ")\n", sep = "")
for (i in seq_along(linguistic_moderators)) {
  cat("    ", i, ". ", linguistic_moderators[i], "\n", sep = "")
}
cat("\n")

cat("  Category 4: Methodological Features (n = ", length(method_moderators), ")\n", sep = "")
for (i in seq_along(method_moderators)) {
  cat("    ", i, ". ", method_moderators[i], "\n", sep = "")
}
cat("\n")

cat("  Category 5: Outcome Characteristics (n = ", length(outcome_moderators), ")\n", sep = "")
for (i in seq_along(outcome_moderators)) {
  cat("    ", i, ". ", outcome_moderators[i], "\n", sep = "")
}
cat("\n")

cat("  ", paste0(rep("-", 76), collapse = ""), "\n", sep = "")
cat("  TOTAL CANDIDATE MODERATORS: ", length(all_moderators), "\n", sep = "")
cat("  ", paste0(rep("-", 76), collapse = ""), "\n", sep = "")

cat("\n")

# Filter to only include moderators present in dataset
available_moderators <- intersect(all_moderators, names(df))
missing_moderators <- setdiff(all_moderators, names(df))

if (length(missing_moderators) > 0) {
  cat("  ⚠️  WARNING: ", length(missing_moderators), " moderator(s) not in dataset:\n", sep = "")
  for (mod in missing_moderators) {
    cat("     - ", mod, "\n", sep = "")
  }
  cat("\n")
}

cat("  ✅ Available moderators: ", length(available_moderators), "/", 
    length(all_moderators), "\n", sep = "")

# Update moderator list
all_moderators <- available_moderators

cat("\n")

# ----------------------------------------------------------------------------
# Step 2.1.2: Iterate Through Moderators
# ----------------------------------------------------------------------------
cat("Step 2.1.2: Running univariate meta-regressions...\n")
cat(paste0(rep("-", 80), collapse = ""), "\n\n")

# Initialize storage
univariate_results_list <- list()
skipped_moderators <- data.frame(
  Moderator = character(),
  Reason = character(),
  stringsAsFactors = FALSE
)
usable_moderators <- character()

for (mod in all_moderators) {
  cat("  Testing moderator: ", mod, "\n", sep = "")
  
  # Create temporary dataset with complete cases for this moderator
  df_temp <- df[!is.na(df[[mod]]) & !is.na(df$Hedges_g) & !is.na(df$vi), ]
  
  # Check sample size
  n_complete <- nrow(df_temp)
  if (n_complete < 5) {
    cat("    ⚠️  SKIPPED: Insufficient data (n = ", n_complete, ")\n\n", sep = "")
    skipped_moderators <- rbind(
      skipped_moderators,
      data.frame(Moderator = mod, Reason = paste0("n < 5 (n=", n_complete, ")"))
    )
    next
  }
  
  # Check number of levels for categorical moderators
  if (is.character(df_temp[[mod]]) || is.factor(df_temp[[mod]])) {
    n_levels <- length(unique(df_temp[[mod]]))
    if (n_levels < 2) {
      cat("    ⚠️  SKIPPED: Insufficient levels (k = ", n_levels, ")\n\n", sep = "")
      skipped_moderators <- rbind(
        skipped_moderators,
        data.frame(Moderator = mod, Reason = paste0("k < 2 (k=", n_levels, ")"))
      )
      next
    }
  }
  
  # Fit univariate meta-regression
  mod_formula <- as.formula(paste("Hedges_g ~", mod))
  
  res_mod <- tryCatch(
    {
      rma(yi = Hedges_g, 
          vi = df_temp$vi,  # Explicitly use dataframe column
          mods = mod_formula, 
          data = df_temp, 
          method = "REML")
    },
    error = function(e) {
      cat("    ❌ ERROR: ", e$message, "\n\n", sep = "")
      return(NULL)
    }
  )
  
  # Check if model fitted successfully
  if (is.null(res_mod)) {
    skipped_moderators <- rbind(
      skipped_moderators,
      data.frame(Moderator = mod, Reason = "Model convergence failure")
    )
    next
  }
  
  # Calculate pseudo-R²
  # R² = (τ²_overall - τ²_moderator) / τ²_overall × 100%
  R2 <- max(0, (tau2 - res_mod$tau2) / tau2 * 100)
  
  # Store results
  univariate_results_list[[mod]] <- list(
    model = res_mod,
    moderator = mod,
    k = res_mod$k,
    QM = res_mod$QM,
    QMp = res_mod$QMp,
    tau2 = res_mod$tau2,
    R2 = R2
  )
  
  usable_moderators <- c(usable_moderators, mod)
  
  # Display summary
  cat("    ✅ Model fitted successfully\n")
  cat("       k = ", res_mod$k, ", QM = ", sprintf("%.4f", res_mod$QM), 
      ", p = ", sprintf("%.4f", res_mod$QMp), "\n", sep = "")
  cat("       τ² = ", sprintf("%.4f", res_mod$tau2), ", R² = ", 
      sprintf("%.2f", R2), "%\n\n", sep = "")
}

# Summary statistics
cat(paste0(rep("-", 80), collapse = ""), "\n")
cat("  SUMMARY:\n")
cat("    Total candidate moderators:  ", length(all_moderators), "\n", sep = "")
cat("    Successfully fitted models:  ", length(usable_moderators), "\n", sep = "")
cat("    Skipped moderators:          ", nrow(skipped_moderators), "\n", sep = "")
cat(paste0(rep("-", 80), collapse = ""), "\n\n")

if (nrow(skipped_moderators) > 0) {
  cat("  Skipped moderators:\n")
  for (i in 1:nrow(skipped_moderators)) {
    cat("    - ", skipped_moderators$Moderator[i], ": ", 
        skipped_moderators$Reason[i], "\n", sep = "")
  }
  cat("\n")
}

# ----------------------------------------------------------------------------
# Step 2.1.3: Create Summary Table
# ----------------------------------------------------------------------------
cat("Step 2.1.3: Creating summary table...\n")
cat(paste0(rep("-", 80), collapse = ""), "\n\n")

# Compile univariate results
univariate_summary <- data.frame(
  Moderator = character(),
  k = integer(),
  QM = numeric(),
  df_QM = integer(),
  p_QM = numeric(),
  tau2 = numeric(),
  R2 = numeric(),
  stringsAsFactors = FALSE
)

for (mod in usable_moderators) {
  res <- univariate_results_list[[mod]]
  
  univariate_summary <- rbind(
    univariate_summary,
    data.frame(
      Moderator = mod,
      k = res$k,
      QM = round(res$QM, 4),
      df_QM = res$model$p - 1,
      p_QM = round(res$QMp, 4),
      tau2 = round(res$tau2, 4),
      R2 = round(res$R2, 2),
      stringsAsFactors = FALSE
    )
  )
}

# Display top 5 moderators by R²
cat("  Top 5 moderators by R²:\n")
top5 <- univariate_summary[order(-univariate_summary$R2), ][1:min(5, nrow(univariate_summary)), ]
for (i in 1:nrow(top5)) {
  cat("    ", i, ". ", top5$Moderator[i], ": R² = ", top5$R2[i], 
      "%, p = ", sprintf("%.4f", top5$p_QM[i]), "\n", sep = "")
}
cat("\n")

# ----------------------------------------------------------------------------
# Step 2.1.4: Export Univariate Results
# ----------------------------------------------------------------------------
cat("Step 2.1.4: Exporting univariate results...\n")
cat(paste0(rep("-", 80), collapse = ""), "\n")

# Create output directory for Step 2
step2_dir <- file.path(getwd(), "Meta_Analysis_Results", "Step2_Moderator_Analysis")
if (!dir.exists(step2_dir)) {
  dir.create(step2_dir, recursive = TRUE)
}

# Write CSV file using safe write function
safe_write_csv(univariate_summary, "univariate_moderator_summary.csv", output_dir = step2_dir)

cat("  ✅ Univariate summary saved to: ", step2_dir, "\n", sep = "")
cat("\n")

# Generate detailed univariate_moderator_results.csv with level-by-level info
cat("Step 2.1.3b: Generating detailed level-by-level results...\n")
cat(paste0(rep("-", 80), collapse = ""), "\n\n")

univariate_results_detailed <- data.frame(
  Moderator = character(),
  Level = character(),
  Type = character(),
  n = integer(),
  Estimate = numeric(),
  SE = numeric(),
  CI_Lower = numeric(),
  CI_Upper = numeric(),
  z_value = numeric(),
  p_value = numeric(),
  QM = numeric(),
  QM_p = numeric(),
  tau2_residual = numeric(),
  I2_residual = numeric(),
  R2 = numeric(),
  stringsAsFactors = FALSE
)

for (mod in usable_moderators) {
  res_mod <- univariate_results_list[[mod]]$model
  mod_name <- mod
  
  # Determine moderator type
  df_temp <- df[!is.na(df[[mod]]) & !is.na(df$Hedges_g) & !is.na(df$vi), ]
  is_categorical <- is.character(df_temp[[mod]]) || is.factor(df_temp[[mod]])
  mod_type <- ifelse(is_categorical, "Categorical", "Continuous")
  
  # Extract model results
  n_coefs <- length(res_mod$b)
  
  if (n_coefs == 0) {
    next  # Skip if no coefficients
  }
  
  omnibus_QM <- res_mod$QM
  omnibus_QMp <- res_mod$QMp
  tau2_res <- res_mod$tau2
  I2_res <- res_mod$I2
  R2_val <- max(0, (tau2 - res_mod$tau2) / tau2 * 100)
  
  # Extract coefficients for each level
  for (i in 1:n_coefs) {
    # Get coefficient name safely
    coef_names <- names(res_mod$b)
    if (is.null(coef_names) || length(coef_names) < i) {
      level_name <- paste0("Level_", i)
    } else {
      level_name <- coef_names[i]
    }
    
    # Determine level label
    if (is_categorical) {
      # For categorical, create meaningful labels
      if (grepl("intrcpt", level_name, ignore.case = TRUE) || i == 1) {
        level_label <- paste0(mod_name, " (Baseline)")
      } else {
        # Extract level name from coefficient name
        level_label <- gsub(paste0("^", mod_name), "", level_name)
        level_label <- trimws(level_label)
        if (level_label == "") {
          level_label <- paste0(mod_name, " Level ", i)
        }
      }
    } else {
      # For continuous, use moderator name
      level_label <- mod_name
    }
    
    # Count studies for this level
    if (is_categorical && !grepl("Baseline", level_label)) {
      # Try to match the level name in the data
      n_studies <- sum(df_temp[[mod]] == level_label, na.rm = TRUE)
      if (n_studies == 0) {
        n_studies <- nrow(df_temp) / n_coefs  # Estimate if can't find exact match
      }
    } else {
      n_studies <- nrow(df_temp)
    }
    
    # Add row to detailed results
    univariate_results_detailed <- rbind(
      univariate_results_detailed,
      data.frame(
        Moderator = mod_name,
        Level = level_label,
        Type = mod_type,
        n = as.integer(n_studies),
        Estimate = round(as.numeric(res_mod$b[i]), 4),
        SE = round(res_mod$se[i], 4),
        CI_Lower = round(res_mod$ci.lb[i], 4),
        CI_Upper = round(res_mod$ci.ub[i], 4),
        z_value = round(res_mod$zval[i], 4),
        p_value = round(res_mod$pval[i], 4),
        QM = round(omnibus_QM, 4),
        QM_p = round(omnibus_QMp, 4),
        tau2_residual = round(tau2_res, 4),
        I2_residual = round(I2_res, 2),
        R2 = round(R2_val, 2),
        stringsAsFactors = FALSE
      )
    )
  }
}

# Save detailed results
if (nrow(univariate_results_detailed) > 0) {
  safe_write_csv(univariate_results_detailed, "univariate_moderator_results.csv", output_dir = step2_dir)
  cat("  ✅ Detailed level-by-level results saved\n")
  cat("     Rows: ", nrow(univariate_results_detailed), "\n", sep = "")
} else {
  cat("  ⚠️  No detailed results generated\n")
}
cat("\n")


# ----------------------------------------------------------------------------
# Step 2.1.5: Identify Significant Moderators (p < .05)
# ----------------------------------------------------------------------------
cat("Step 2.1.7: Identifying significant moderators (p < .05)...\n")
cat(paste0(rep("-", 80), collapse = ""), "\n\n")

# Filter for significant moderators
significant_moderators <- univariate_summary[univariate_summary$p_QM < 0.05, ]

cat("  Found ", nrow(significant_moderators), " significant moderator(s):\n\n", sep = "")

if (nrow(significant_moderators) > 0) {
  for (i in 1:nrow(significant_moderators)) {
    row <- significant_moderators[i, ]
    cat("    ", i, ". ", row$Moderator, "\n", sep = "")
    cat("       QM = ", sprintf("%.4f", row$QM), " (df = ", row$df_QM, 
        ", p = ", sprintf("%.4f", row$p_QM), ")\n", sep = "")
    cat("       τ² = ", sprintf("%.4f", row$tau2), ", R² = ", 
        sprintf("%.2f", row$R2), "%\n\n", sep = "")
  }
} else {
  cat("    (none)\n\n")
}

# Store significant moderator names for multivariate analysis
significant_mod_names <- significant_moderators$Moderator

cat("  ✅ Significant moderators stored for multivariate analysis\n")
cat("\n")

cat("✅ STEP 2.1 COMPLETE: Univariate moderator analysis complete\n")
cat(paste0(rep("=", 80), collapse = ""), "\n\n")

# ----------------------------------------------------------------------------
# Step 2.2: Multivariate Moderator Analysis
# ----------------------------------------------------------------------------
cat("================================================================================\n")
cat("STEP 2.2: MULTIVARIATE MODERATOR ANALYSIS\n")
cat("================================================================================\n\n")

# Check if there are significant moderators
if (length(significant_mod_names) == 0) {
  cat("  ⚠️  No significant moderators found (all p ≥ .05)\n")
  cat("  ⚠️  Skipping multivariate analysis\n\n")
  
  cat("✅ STEP 2.2 COMPLETE: Multivariate analysis skipped\n")
  cat(paste0(rep("=", 80), collapse = ""), "\n\n")
  
  run_multivariate <- FALSE
} else {
  run_multivariate <- TRUE
  
  # ----------------------------------------------------------------------------
  # Step 2.2.1: Define Multivariate Model
  # ----------------------------------------------------------------------------
  cat("Step 2.2.1: Building multivariate model...\n")
  cat(paste0(rep("-", 80), collapse = ""), "\n\n")
  
  cat("  Including ", length(significant_mod_names), " significant moderator(s):\n", sep = "")
  for (i in seq_along(significant_mod_names)) {
    cat("    ", i, ". ", significant_mod_names[i], "\n", sep = "")
  }
  cat("\n")
  
  # Create formula
  formula_str <- paste("Hedges_g ~", paste(significant_mod_names, collapse = " + "))
  multi_formula <- as.formula(formula_str)
  
  cat("  Formula: ", formula_str, "\n\n", sep = "")
  
  # ----------------------------------------------------------------------------
  # Step 2.2.2: Prepare Dataset
  # ----------------------------------------------------------------------------
  cat("Step 2.2.2: Preparing dataset...\n")
  cat(paste0(rep("-", 80), collapse = ""), "\n")
  
  # Create complete-case dataset
  required_vars <- c("Hedges_g", "vi", significant_mod_names)
  df_multi <- df[complete.cases(df[, required_vars]), ]
  
  cat("  Original dataset:     ", nrow(df), " rows\n", sep = "")
  cat("  Complete-case data:   ", nrow(df_multi), " rows\n", sep = "")
  cat("  Missing data removed: ", nrow(df) - nrow(df_multi), " rows\n\n", sep = "")
  
  # Check sample size
  if (nrow(df_multi) < 10) {
    cat("  ⚠️  WARNING: Very small sample size (n = ", nrow(df_multi), ")\n", sep = "")
    cat("  ⚠️  Results may be unreliable\n\n")
  }
  
  # ----------------------------------------------------------------------------
  # Step 2.2.3: Fit Multivariate Model
  # ----------------------------------------------------------------------------
  cat("Step 2.2.3: Fitting multivariate meta-regression...\n")
  cat(paste0(rep("-", 80), collapse = ""), "\n")
  
  res_multi <- tryCatch(
    {
      rma(yi = Hedges_g, 
          vi = vi, 
          mods = multi_formula, 
          data = df_multi, 
          method = "REML")
    },
    error = function(e) {
      cat("  ❌ ERROR: Model fitting failed\n")
      cat("     Message: ", e$message, "\n\n", sep = "")
      return(NULL)
    }
  )
  
  # Check if model fitted successfully
  if (is.null(res_multi)) {
    cat("  ⚠️  Multivariate model could not be fitted\n")
    cat("  ⚠️  Possible reasons:\n")
    cat("       - Too many moderators relative to sample size\n")
    cat("       - Multicollinearity among moderators\n")
    cat("       - Convergence issues\n\n")
    
    run_multivariate <- FALSE
    
  } else {
    cat("  ✅ Model fitted successfully\n\n")
    
    # ----------------------------------------------------------------------------
    # Step 2.2.4: Display Multivariate Results
    # ----------------------------------------------------------------------------
    cat("Step 2.2.4: Multivariate meta-regression results...\n")
    cat(paste0(rep("-", 80), collapse = ""), "\n\n")
    
    # Model-level statistics
    cat("  MODEL-LEVEL STATISTICS:\n")
    cat("  ", paste0(rep("-", 76), collapse = ""), "\n", sep = "")
    cat("    k (studies):         ", res_multi$k, "\n", sep = "")
    cat("    QM (omnibus test):   ", sprintf("%.4f", res_multi$QM), 
        " (df = ", res_multi$p - 1, ", p = ", sprintf("%.4f", res_multi$QMp), ")\n", sep = "")
    cat("    QE (residual het.):  ", sprintf("%.4f", res_multi$QE), 
        " (df = ", res_multi$k - res_multi$p, ", p = ", sprintf("%.4f", res_multi$QEp), ")\n", sep = "")
    cat("    τ² (residual):       ", sprintf("%.4f", res_multi$tau2), "\n", sep = "")
    cat("    I² (residual):       ", sprintf("%.2f", res_multi$I2), "%\n", sep = "")
    
    # Calculate pseudo-R²
    # R² = (τ²_baseline - τ²_multivariate) / τ²_baseline
    R2_multi <- max(0, (tau2 - res_multi$tau2) / tau2 * 100)
    cat("    Pseudo-R²:           ", sprintf("%.2f", R2_multi), "%\n", sep = "")
    
    cat("\n")
    
    # Coefficient table
    cat("  REGRESSION COEFFICIENTS:\n")
    cat("  ", paste0(rep("=", 76), collapse = ""), "\n", sep = "")
    cat("\n")
    
    coef_table <- data.frame(
      Moderator = rownames(res_multi$b),
      Estimate = as.numeric(res_multi$b),
      SE = res_multi$se,
      CI_Lower = res_multi$ci.lb,
      CI_Upper = res_multi$ci.ub,
      z_value = res_multi$zval,
      p_value = res_multi$pval,
      stringsAsFactors = FALSE
    )
    
    for (i in 1:nrow(coef_table)) {
      row <- coef_table[i, ]
      
      # Determine significance
      if (row$p_value < 0.001) {
        sig_marker <- "***"
        sig_level <- "p < .001"
      } else if (row$p_value < 0.01) {
        sig_marker <- "**"
        sig_level <- "p < .01"
      } else if (row$p_value < 0.05) {
        sig_marker <- "*"
        sig_level <- "p < .05"
      } else {
        sig_marker <- ""
        sig_level <- "n.s."
      }
      
      cat("  ", row$Moderator, " ", sig_marker, "\n", sep = "")
      cat("    β = ", sprintf("%.4f", row$Estimate), 
          " [", sprintf("%.4f", row$CI_Lower), ", ", 
          sprintf("%.4f", row$CI_Upper), "]\n", sep = "")
      cat("    z = ", sprintf("%.4f", row$z_value), 
          ", p = ", sprintf("%.4f", row$p_value), " (", sig_level, ")\n", sep = "")
      cat("\n")
    }
    
    # ----------------------------------------------------------------------------
    # Step 2.2.5: Export Multivariate Results (SAFE VERSION)
    # ----------------------------------------------------------------------------
    cat("Step 2.2.5: Exporting multivariate results...\n")
    cat(paste0(rep("-", 80), collapse = ""), "\n\n")
    
    # Get coefficient names from the fitted model
    coef_names <- rownames(res_multi$b)
    
    cat("  Raw coefficient names from model:\n")
    for (i in seq_along(coef_names)) {
      cat("    ", i, ". ", coef_names[i], "\n", sep = "")
    }
    cat("\n")
    
    # ----------------------------------------------------------------------------
    # SAFE PARSING LOGIC
    # ----------------------------------------------------------------------------
    # Strategy:
    # 1. Get model.matrix to identify TRUE variable types
    # 2. For each coefficient, determine if it's:
    #    - Intercept
    #    - Continuous moderator (numeric variable)
    #    - Categorical level (factor dummy variable)
    # 3. NEVER assume coef_name == mod means anything - check actual data type
    # ----------------------------------------------------------------------------
    
    # Get model matrix to understand variable structure
    X <- model.matrix(res_multi)
    
    # Identify moderator types from original data
    continuous_mods <- character()
    categorical_mods <- character()
    
    for (mod in significant_mod_names) {
      if (is.numeric(df_multi[[mod]])) {
        continuous_mods <- c(continuous_mods, mod)
      } else {
        categorical_mods <- c(categorical_mods, mod)
      }
    }
    
    cat("  Moderator type detection:\n")
    cat("    Continuous moderators (n=", length(continuous_mods), "):\n", sep = "")
    for (mod in continuous_mods) {
      cat("      - ", mod, "\n", sep = "")
    }
    cat("    Categorical moderators (n=", length(categorical_mods), "):\n", sep = "")
    for (mod in categorical_mods) {
      cat("      - ", mod, "\n", sep = "")
    }
    cat("\n")
    
    # Parse each coefficient
    parsed_coefficients <- list()
    
    for (coef_name in coef_names) {
      # Handle intercept
      if (coef_name == "intrcpt" || coef_name == "(Intercept)") {
        parsed_coefficients[[length(parsed_coefficients) + 1]] <- list(
          original_name = coef_name,
          moderator_family = "Intercept",
          level_label = "Intercept",
          type = "intercept"
        )
        next
      }
      
      # Check if this is a continuous moderator
      # For continuous: coefficient name == moderator name (exact match)
      if (coef_name %in% continuous_mods) {
        parsed_coefficients[[length(parsed_coefficients) + 1]] <- list(
          original_name = coef_name,
          moderator_family = coef_name,
          level_label = coef_name,  # Use full name as label
          type = "continuous"
        )
        next
      }
      
      # For categorical moderators: try to match moderator family
      matched <- FALSE
      for (cat_mod in categorical_mods) {
        # Check if coefficient name contains this categorical moderator
        # e.g., "L1Arabic" contains "L1", "Education_StageUndergraduate" contains "Education_Stage"
        if (grepl(paste0("^", cat_mod), coef_name)) {
          # Extract level label by removing moderator prefix
          level_label <- sub(paste0("^", cat_mod), "", coef_name)
          
          # CRITICAL: If level_label is empty after removal, this means
          # coef_name == cat_mod, which should NEVER happen for categorical moderators
          # This would be a parsing error - skip it
          if (level_label == "" || nchar(level_label) == 0) {
            cat("  ⚠️  WARNING: Skipping invalid coefficient '", coef_name, 
                "' (categorical moderator without level)\n", sep = "")
            matched <- TRUE
            break
          }
          
          parsed_coefficients[[length(parsed_coefficients) + 1]] <- list(
            original_name = coef_name,
            moderator_family = cat_mod,
            level_label = level_label,
            type = "categorical"
          )
          matched <- TRUE
          break
        }
      }
      
      # If no match found, this is an unknown coefficient - skip it
      if (!matched) {
        cat("  ⚠️  WARNING: Could not parse coefficient '", coef_name, "' - skipping\n", sep = "")
      }
    }
    
    cat("\n  Parsed ", length(parsed_coefficients), " valid coefficients\n\n", sep = "")
    
    # Create export dataframe
    coef_export <- data.frame(
      Moderator_Family = character(),
      Level_Label = character(),
      Estimate = numeric(),
      SE = numeric(),
      CI_Lower = numeric(),
      CI_Upper = numeric(),
      z_value = numeric(),
      p_value = numeric(),
      stringsAsFactors = FALSE
    )
    
    # Add parsed coefficients to export
    for (i in seq_along(parsed_coefficients)) {
      parsed <- parsed_coefficients[[i]]
      
      # Find matching row in res_multi results
      coef_idx <- which(coef_names == parsed$original_name)
      
      coef_export <- rbind(coef_export, data.frame(
        Moderator_Family = parsed$moderator_family,
        Level_Label = parsed$level_label,
        Estimate = round(as.numeric(res_multi$b[coef_idx]), 4),
        SE = round(res_multi$se[coef_idx], 4),
        CI_Lower = round(res_multi$ci.lb[coef_idx], 4),
        CI_Upper = round(res_multi$ci.ub[coef_idx], 4),
        z_value = round(res_multi$zval[coef_idx], 4),
        p_value = round(res_multi$pval[coef_idx], 4),
        stringsAsFactors = FALSE
      ))
    }
    
    # ----------------------------------------------------------------------------
    # Add baseline rows for categorical moderators
    # ----------------------------------------------------------------------------
    cat("  Adding baseline rows for categorical moderators...\n")
    
    # Get unique categorical moderators that appear in the export
    categorical_mods_in_export <- unique(coef_export$Moderator_Family[
      coef_export$Moderator_Family != "Intercept" &
      coef_export$Moderator_Family %in% categorical_mods
    ])
    
    baseline_rows <- list()
    
    for (cat_mod in categorical_mods_in_export) {
      # Get baseline from BASELINE_LEVELS
      baseline_level <- NULL
      
      # Try BASELINE_<Moderator> variable
      baseline_var_name <- paste0("BASELINE_", cat_mod)
      if (exists(baseline_var_name)) {
        baseline_level <- get(baseline_var_name)
      } else if (exists("BASELINE_LEVELS") && cat_mod %in% names(BASELINE_LEVELS)) {
        baseline_level <- BASELINE_LEVELS[[cat_mod]]
      }
      
      # Only add baseline if we found it
      if (!is.null(baseline_level)) {
        baseline_rows[[length(baseline_rows) + 1]] <- data.frame(
          Moderator_Family = cat_mod,
          Level_Label = baseline_level,
          Estimate = 0.0000,
          SE = 0.0000,
          CI_Lower = 0.0000,
          CI_Upper = 0.0000,
          z_value = 0.0000,
          p_value = 1.0000,
          stringsAsFactors = FALSE
        )
        
        cat("    ✅ Added baseline for ", cat_mod, ": ", baseline_level, "\n", sep = "")
      } else {
        cat("    ⚠️  No baseline found for ", cat_mod, "\n", sep = "")
      }
    }
    
    cat("\n")
    
    # Combine with baseline rows
    if (length(baseline_rows) > 0) {
      coef_export <- rbind(coef_export, do.call(rbind, baseline_rows))
      
      # Sort: Intercept first, then by Moderator_Family, then by level
      # Within each family: contrasts first, baseline last (Estimate=0)
      coef_export <- coef_export[order(
        -(coef_export$Moderator_Family == "Intercept"),  # Intercept first
        coef_export$Moderator_Family,                     # Alphabetical by family
        -(coef_export$Estimate == 0),                     # Baseline last within family
        coef_export$Level_Label                           # Alphabetical by level
      ), ]
    }
    
    # Display final export structure
    cat("  Final export structure:\n")
    cat("    Total rows:       ", nrow(coef_export), "\n", sep = "")
    cat("    Intercept:        ", sum(coef_export$Moderator_Family == "Intercept"), "\n", sep = "")
    cat("    Continuous:       ", sum(coef_export$Moderator_Family %in% continuous_mods & 
                                       coef_export$Estimate != 0), "\n", sep = "")
    cat("    Categorical:      ", sum(coef_export$Moderator_Family %in% categorical_mods & 
                                       coef_export$Estimate != 0), "\n", sep = "")
    cat("    Baselines:        ", sum(coef_export$Estimate == 0 & 
                                       coef_export$Moderator_Family != "Intercept"), "\n\n", sep = "")
    
    # Model fit statistics
    model_fit <- data.frame(
      Statistic = c("k", "QM", "QM_df", "QM_p", "QE", "QE_df", "QE_p", 
                    "tau2", "I2", "Pseudo_R2"),
      Value = c(
        res_multi$k,
        round(res_multi$QM, 4),
        res_multi$p - 1,
        round(res_multi$QMp, 4),
        round(res_multi$QE, 4),
        res_multi$k - res_multi$p,
        round(res_multi$QEp, 4),
        round(res_multi$tau2, 4),
        round(res_multi$I2, 2),
        round(R2_multi, 2)
      ),
      stringsAsFactors = FALSE
    )
    
    # Verify step2_dir exists
    if (!exists("step2_dir") || is.null(step2_dir)) {
      step2_dir <- file.path(getwd(), "Meta_Analysis_Results", "Step2_Moderator_Analysis")
      if (!dir.exists(step2_dir)) dir.create(step2_dir, recursive = TRUE)
    }
    
    # Write CSV files to Step2 folder
    safe_write_csv(coef_export, "multivariate_model_coefficients.csv", output_dir = step2_dir)
    safe_write_csv(model_fit, "multivariate_model_fit.csv", output_dir = step2_dir)
    
    cat("  ✅ Exported coefficients with safe parsing logic\n")
    cat("  ✅ Format: Moderator_Family + Level_Label\n")
    cat("  ✅ Categorical moderators: Family + Level (baselines added with β=0)\n")
    cat("  ✅ Continuous moderators: Family = Level = moderator name\n")
    
    cat("\n")
  }
}

if (!run_multivariate) {
  cat("  ⚠️  Multivariate analysis not performed\n\n")
  cat("✅ STEP 2.2 COMPLETE: Multivariate analysis not performed\n")
}
cat(paste0(rep("=", 80), collapse = ""), "\n\n")


###############################################################################
#                   STEP 2: MODERATOR META-ANALYSIS                          #
###############################################################################

Verifying STEP 1 dependencies...
-------------------------------------------------------------------------------- 
  ✅ tau2 found: τ² = 0.08823
  ✅ res_overall found: k = 29

STEP 2.1: UNIVARIATE MODERATOR ANALYSIS

Step 2.1.1: Defining moderator variables...
-------------------------------------------------------------------------------- 

  Category 1: Participant Characteristics (n = 5)
    1. L1
    2. Age_Group
    3. Proficiency_Level
    4. English_Major
    5. Education_Stage

  Category 2: Treatment Characteristics (n = 7)
    1. Treatment_Duration
    2. Training_TotalMinute
    3. Training_TotalWeeks
    4. Training_Context
    5. Feedback_Type
    6. Visual_Cue
    7. Instructor_Type

  Category 3: Linguistic Features (n = 2)
    1. Targe

[1] TRUE

  ✅ Univariate summary saved to: c:/Users/Lenovo/Desktop/github/Meta_Analysis_Results/Step2_Moderator_Analysis

Step 2.1.3b: Generating detailed level-by-level results...
-------------------------------------------------------------------------------- 

  ✅ Saved: univariate_moderator_results.csv
  ✅ Detailed level-by-level results saved
     Rows: 63

Step 2.1.7: Identifying significant moderators (p < .05)...
-------------------------------------------------------------------------------- 

  Found 4 significant moderator(s):

    1. L1
       QM = 22.5717 (df = 7, p = 0.0020)
       τ² = 0.0248, R² = 71.94%

    2. English_Major
       QM = 12.4787 (df = 2, p = 0.0020)
       τ² = 0.0386, R² = 56.26%

    3. Education_Stage
       QM = 6.9669 (df = 2, p = 0.0307)
       τ² = 0.0642, R² = 27.22%

    4. Treatment_Duration
       QM = 10.7672 (df = 3, p = 0.0131)
       τ² = 0.0485, R² = 44.98%

  ✅ Significant moderators stored for multivariate analysis

✅ STEP 2.1 COMPLETE: Univaria

Warning message:
"Redundant predictors dropped from the model."


  ✅ Model fitted successfully

Step 2.2.4: Multivariate meta-regression results...
-------------------------------------------------------------------------------- 

  MODEL-LEVEL STATISTICS:
  ----------------------------------------------------------------------------
    k (studies):         29
    QM (omnibus test):   36.8227 (df = 13, p = 0.0004)
    QE (residual het.):  19.8100 (df = 15, p = 0.1793)
    τ² (residual):       0.0075
    I² (residual):       9.71%
    Pseudo-R²:           91.47%

  REGRESSION COEFFICIENTS:

  intrcpt 
    β = -0.4689 [-1.1805, 0.2426]
    z = -1.2917, p = 0.1965 (n.s.)

  L1 
    β = 0.3633 [-0.5723, 1.2990]
    z = 0.7611, p = 0.4466 (n.s.)

  L1Chinese *
    β = 0.6163 [0.0344, 1.1983]
    z = 2.0758, p = 0.0379 (p < .05)

  L1Farsi **
    β = 1.4003 [0.4293, 2.3714]
    z = 2.8265, p = 0.0047 (p < .01)

  L1Japanese 
    β = 0.4056 [-0.2812, 1.0923]
    z = 1.1575, p = 0.2471 (n.s.)

  L1Korean 
    β = 0.6539 [-0.5248, 1.8325]
    z = 1.0873, p 

In [9]:
# ============================================================================
# STEP 3: SENSITIVITY & ROBUSTNESS ANALYSES (LEAVE-ONE-OUT, INFLUENCE, RVE)
# ============================================================================
#
# Outputs (All CSV files):
#   STEP 3.1 — Leave-one-out Sensitivity:
#       • leave_one_out_analysis.csv
#
#   STEP 3.2 — Influence Diagnostics:
#       • influence_diagnostics.csv
#
#   STEP 3.3 — Robust Variance Estimation (RVE):
#       • rve_overall_effect.csv
#       • rve_moderator_results.csv          (only if significant moderators exist)
#
# ============================================================================


# ============================================================================
# STEP 3.1: LEAVE-ONE-OUT SENSITIVITY ANALYSIS
# ============================================================================
#
# Purpose:  Test robustness of pooled effect to individual study exclusion
# Method:   Iteratively refit model with k-1 studies (leave1out)
#
# Influence Criteria:
#   • |Δg| > 10% (estimate changes >10%)
#   • Significance flip (p crosses .05 threshold)
#   • ΔI² > 10% (heterogeneity shift)
#
# Outputs:
#   Effect estimate range, influential cases, stability classification
#
# Interpretation:
#   Narrow range (Δg <5%) → Highly robust
#   Moderate range (5-10%) → Robust
#   Wide range (>10%) → Potentially fragile, driven by specific studies
#
# ============================================================================

cat(paste0(rep("=", 80), collapse = ""), "\n")
cat("STEP 3.1: LEAVE-ONE-OUT SENSITIVITY ANALYSIS\n")
cat(paste0(rep("=", 80), collapse = ""), "\n\n")

cat("Research Question: Is the overall effect robust to study exclusion?\n\n")

# ----------------------------------------------------------------------------
# Step 3.1.1: Review Overall Effect (Baseline)
# ----------------------------------------------------------------------------
cat("Step 3.1.1: Baseline overall effect (for comparison)...\n")
cat(paste0(rep("-", 80), collapse = ""), "\n\n")

cat("  BASELINE (All studies included):\n")
cat("    Effect size:  g = ", sprintf("%.4f", overall_g), 
    " [", sprintf("%.4f", overall_ci_lb), ", ",
    sprintf("%.4f", overall_ci_ub), "]\n", sep = "")
cat("    p-value:      ", sprintf("%.4f", overall_p), sep = "")
if (overall_p < 0.001) {
  cat(" ***\n")
} else if (overall_p < 0.01) {
  cat(" **\n")
} else if (overall_p < 0.05) {
  cat(" *\n")
} else {
  cat("\n")
}
cat("    τ²:           ", sprintf("%.4f", tau2), "\n", sep = "")
cat("    I²:           ", sprintf("%.2f", I2), "%\n", sep = "")
cat("    k:            ", res_overall$k, " effect sizes\n", sep = "")

cat("\n")

# ----------------------------------------------------------------------------
# Step 3.1.2: Perform Leave-One-Out Analysis
# ----------------------------------------------------------------------------
cat("Step 3.1.2: Running leave-one-out analysis...\n")
cat(paste0(rep("-", 80), collapse = ""), "\n")

cat("  Systematically excluding each effect size...\n\n")

# Use metafor's built-in function
loo_results <- leave1out(res_overall)

cat("  ✅ Leave-one-out analysis completed\n")
cat("     Iterations: ", nrow(loo_results), "\n", sep = "")

cat("\n")

# ----------------------------------------------------------------------------
# Step 3.1.3: Organize Leave-One-Out Results
# ----------------------------------------------------------------------------
cat("Step 3.1.3: Organizing sensitivity results...\n")
cat(paste0(rep("-", 80), collapse = ""), "\n")

# Create comprehensive results dataframe
loo_summary <- data.frame(
  Study_ID = df$Study_ID,
  Effect_ID = df$Effect_ID,
  Estimate = loo_results$estimate,
  SE = loo_results$se,
  CI_Lower = loo_results$ci.lb,
  CI_Upper = loo_results$ci.ub,
  z_value = loo_results$zval,
  p_value = loo_results$pval,
  Q = loo_results$Q,
  Q_p = loo_results$Qp,
  tau2 = loo_results$tau2,
  I2 = loo_results$I2,
  H2 = loo_results$H2,
  stringsAsFactors = FALSE
)

# Calculate change metrics
loo_summary$Change_Estimate <- loo_summary$Estimate - overall_g
loo_summary$Pct_Change_Estimate <- (loo_summary$Change_Estimate / overall_g) * 100
loo_summary$Change_I2 <- loo_summary$I2 - I2

cat("  ✅ Results organized with change metrics\n")

cat("\n")

# ----------------------------------------------------------------------------
# Step 3.1.4: Summary Statistics
# ----------------------------------------------------------------------------
cat("Step 3.1.4: Summary statistics across leave-one-out iterations...\n")
cat(paste0(rep("-", 80), collapse = ""), "\n\n")

cat("  EFFECT SIZE RANGE WHEN EACH STUDY EXCLUDED:\n")
cat("  ", paste0(rep("-", 76), collapse = ""), "\n", sep = "")
cat("    Minimum:  ", sprintf("%.4f", min(loo_summary$Estimate)), "\n", sep = "")
cat("    Maximum:  ", sprintf("%.4f", max(loo_summary$Estimate)), "\n", sep = "")
cat("    Range:    ", sprintf("%.4f", max(loo_summary$Estimate) - min(loo_summary$Estimate)), "\n", sep = "")
cat("    Mean:     ", sprintf("%.4f", mean(loo_summary$Estimate)), "\n", sep = "")
cat("    Median:   ", sprintf("%.4f", median(loo_summary$Estimate)), "\n", sep = "")
cat("    SD:       ", sprintf("%.4f", sd(loo_summary$Estimate)), "\n", sep = "")

cat("\n")

cat("  I² RANGE:\n")
cat("  ", paste0(rep("-", 76), collapse = ""), "\n", sep = "")
cat("    Minimum:  ", sprintf("%.2f", min(loo_summary$I2)), "%\n", sep = "")
cat("    Maximum:  ", sprintf("%.2f", max(loo_summary$I2)), "%\n", sep = "")
cat("    Range:    ", sprintf("%.2f", max(loo_summary$I2) - min(loo_summary$I2)), "%\n", sep = "")

cat("\n")

# ----------------------------------------------------------------------------
# Step 3.1.5: Identify Influential Studies
# ----------------------------------------------------------------------------
cat("Step 3.1.5: Identifying influential studies...\n")
cat(paste0(rep("-", 80), collapse = ""), "\n\n")

# Define influence criteria
influence_threshold_estimate <- 10  # 10% change in estimate
influence_threshold_I2 <- 10        # 10 percentage point change in I²

# Flag influential cases
loo_summary$Influential_Estimate <- abs(loo_summary$Pct_Change_Estimate) > influence_threshold_estimate
loo_summary$Influential_I2 <- abs(loo_summary$Change_I2) > influence_threshold_I2
loo_summary$Influential_Any <- loo_summary$Influential_Estimate | loo_summary$Influential_I2

# Check for significance changes
loo_summary$Sig_Original <- overall_p < 0.05
loo_summary$Sig_LOO <- loo_summary$p_value < 0.05
loo_summary$Sig_Changed <- loo_summary$Sig_Original != loo_summary$Sig_LOO

# Count influential cases
n_influential_estimate <- sum(loo_summary$Influential_Estimate)
n_influential_I2 <- sum(loo_summary$Influential_I2)
n_influential_any <- sum(loo_summary$Influential_Any)
n_sig_changed <- sum(loo_summary$Sig_Changed)

cat("  INFLUENCE DETECTION:\n")
cat("  ", paste0(rep("=", 76), collapse = ""), "\n", sep = "")
cat("\n")

if (n_influential_any > 0) {
  cat("  ⚠️  INFLUENTIAL STUDIES DETECTED: ", n_influential_any, "/", nrow(loo_summary), "\n\n", sep = "")
  
  # Show influential studies
  influential_studies <- loo_summary[loo_summary$Influential_Any, ]
  influential_studies <- influential_studies[order(-abs(influential_studies$Pct_Change_Estimate)), ]
  
  for (i in 1:nrow(influential_studies)) {
    row <- influential_studies[i, ]
    cat("  ", i, ". ", row$Study_ID, " (", row$Effect_ID, ")\n", sep = "")
    cat("     When excluded: g = ", sprintf("%.4f", row$Estimate), 
        " (change: ", sprintf("%+.2f", row$Pct_Change_Estimate), "%)\n", sep = "")
    cat("     I² change: ", sprintf("%+.2f", row$Change_I2), " percentage points\n", sep = "")
    
    if (row$Influential_Estimate) {
      cat("     ⚠️  Large effect on estimate (>10%)\n")
    }
    if (row$Influential_I2) {
      cat("     ⚠️  Large effect on heterogeneity (ΔI² >10%)\n")
    }
    cat("\n")
  }
  
} else {
  cat("  ✅ NO HIGHLY INFLUENTIAL STUDIES DETECTED\n")
  cat("     All individual exclusions change estimate by <10%\n")
  cat("     All individual exclusions change I² by <10 percentage points\n\n")
}

# Check significance stability
if (n_sig_changed > 0) {
  cat("  ⚠️  SIGNIFICANCE INSTABILITY DETECTED\n")
  cat("     ", n_sig_changed, " study/studies alter statistical significance\n\n", sep = "")
  
  sig_changed_studies <- loo_summary[loo_summary$Sig_Changed, ]
  for (i in 1:nrow(sig_changed_studies)) {
    row <- sig_changed_studies[i, ]
    cat("     - ", row$Study_ID, " (", row$Effect_ID, "): ", sep = "")
    cat("p = ", sprintf("%.4f", row$p_value), "\n", sep = "")
  }
  cat("\n")
} else {
  cat("  ✅ SIGNIFICANCE IS STABLE\n")
  cat("     Statistical significance consistent across all iterations\n\n")
}

# ----------------------------------------------------------------------------
# Step 3.1.6: Interpretation and Conclusion
# ----------------------------------------------------------------------------
cat("Step 3.1.6: Sensitivity analysis interpretation...\n")
cat(paste0(rep("-", 80), collapse = ""), "\n\n")

cat("  ROBUSTNESS ASSESSMENT:\n")
cat("  ", paste0(rep("=", 76), collapse = ""), "\n", sep = "")
cat("\n")

# Determine robustness level
if (n_influential_any == 0 && n_sig_changed == 0) {
  robustness_level <- "HIGHLY ROBUST"
  robustness_desc <- "Results are stable across all leave-one-out iterations"
  robustness_icon <- "✅"
} else if (n_influential_any <= 2 && n_sig_changed == 0) {
  robustness_level <- "ROBUST"
  robustness_desc <- "Minimal influence from individual studies"
  robustness_icon <- "✅"
} else if (n_sig_changed == 0) {
  robustness_level <- "MODERATELY ROBUST"
  robustness_desc <- "Some individual influence but significance is stable"
  robustness_icon <- "⚠️ "
} else {
  robustness_level <- "FRAGILE"
  robustness_desc <- "Results depend on specific studies"
  robustness_icon <- "⚠️ "
}

cat("  ", robustness_icon, " CONCLUSION: ", robustness_level, "\n", sep = "")
cat("     ", robustness_desc, "\n\n", sep = "")

cat("  RECOMMENDATION:\n")
if (n_influential_any == 0) {
  cat("     → Report overall effect with confidence\n")
  cat("     → No sensitivity concerns\n")
} else if (n_influential_any <= 2) {
  cat("     → Report overall effect\n")
  cat("     → Note influential studies in supplementary materials\n")
} else {
  cat("     → Report overall effect with caution\n")
  cat("     → Conduct influence diagnostics (STEP 3.2)\n")
  cat("     → Consider excluding outliers in sensitivity analysis\n")
}

cat("\n")

# ----------------------------------------------------------------------------
# Step 3.1.7: Export Leave-One-Out Results
# ----------------------------------------------------------------------------
cat("Step 3.1.7: Exporting leave-one-out results...\n")
cat(paste0(rep("-", 80), collapse = ""), "\n")

# Round for export
loo_export <- loo_summary
numeric_cols <- c("Estimate", "SE", "CI_Lower", "CI_Upper", "z_value", "p_value",
                  "Q", "Q_p", "tau2", "I2", "H2", "Change_Estimate", 
                  "Pct_Change_Estimate", "Change_I2")
for (col in numeric_cols) {
  if (col %in% names(loo_export)) {
    loo_export[[col]] <- round(loo_export[[col]], 4)
  }
}

# Verify step3_dir exists
if (!exists("step3_dir") || is.null(step3_dir)) {
  step3_dir <- file.path(getwd(), "Meta_Analysis_Results", "Step3_Robustness_Checks")
  if (!dir.exists(step3_dir)) dir.create(step3_dir, recursive = TRUE)
}

# Save to Step3 folder
safe_write_csv(loo_export, "leave_one_out_analysis.csv", output_dir = step3_dir)

cat("\n")
cat("✅ STEP 3.1 COMPLETE: Leave-one-out sensitivity analysis finished\n")
cat(paste0(rep("=", 80), collapse = ""), "\n\n")



# ============================================================================
# STEP 3.2: INFLUENCE DIAGNOSTICS
# ============================================================================
#
# Purpose:  Detect outliers and high-leverage cases via formal diagnostics
# Metrics:  Cook's D, DFBETAS, Hat values, Standardized residuals
#
# Thresholds (established criteria):
#   Cook's D:    Di > 4/(k-p-1)
#   DFBETAS:     |DFBETAS| > 2/√k
#   Hat values:  hi > 2p/k (leverage)
#   Residuals:   |ri| > 2 (moderate), |ri| > 3 (extreme)
#
# Outputs:
#   Flagged influential cases, sensitivity analysis excluding outliers
#
# Interpretation:
#   Cases exceeding multiple thresholds warrant exclusion sensitivity tests
#   Compare results with/without influential cases for robustness assessment
#
# ============================================================================

cat(paste0(rep("=", 80), collapse = ""), "\n")
cat("STEP 3.2: INFLUENCE DIAGNOSTICS\n")
cat(paste0(rep("=", 80), collapse = ""), "\n\n")

cat("Research Question: Are there influential outliers affecting results?\n\n")

# ----------------------------------------------------------------------------
# Step 3.2.1: Calculate Influence Diagnostics
# ----------------------------------------------------------------------------
cat("Step 3.2.1: Calculating influence diagnostics...\n")
cat(paste0(rep("-", 80), collapse = ""), "\n")

cat("  Computing diagnostic metrics:\n")
cat("    → Cook's Distance (overall influence)\n")
cat("    → DFBETAS (coefficient influence)\n")
cat("    → Hat values (leverage)\n")
cat("    → Standardized residuals (outliers)\n\n")

# Use metafor's influence function
influence_stats <- influence(res_overall)

cat("  ✅ Diagnostic calculations complete\n")

cat("\n")

# ----------------------------------------------------------------------------
# Step 3.2.2: Extract and Organize Diagnostics
# ----------------------------------------------------------------------------
cat("Step 3.2.2: Organizing diagnostic statistics...\n")
cat(paste0(rep("-", 80), collapse = ""), "\n")

# Extract diagnostics
cooks_d <- influence_stats$inf$cook.d        # Cook's distance
dfbetas_raw <- influence_stats$inf$dfbs      # DFBETAS (may be matrix or NULL)
hat_values <- influence_stats$inf$hat        # Hat values (leverage)
std_resid_obj <- rstandard(res_overall)      # Standardized residuals object

# Extract standardized residuals
if (is.list(std_resid_obj) && "z" %in% names(std_resid_obj)) {
  std_resid <- std_resid_obj$z
} else if (is.numeric(std_resid_obj)) {
  std_resid <- std_resid_obj
} else {
  std_resid <- as.numeric(std_resid_obj)
}

# Handle DFBETAS - can be matrix, vector, or NULL
if (is.null(dfbetas_raw) || length(dfbetas_raw) == 0) {
  # If DFBETAS not available, calculate it manually
  dfbetas <- rep(0, length(cooks_d))
  cat("  ⚠️  DFBETAS not available from influence(), using zeros\n")
} else if (is.matrix(dfbetas_raw)) {
  # For each case, take maximum absolute DFBETAS across all coefficients
  dfbetas <- apply(abs(dfbetas_raw), 1, max)
} else if (is.numeric(dfbetas_raw)) {
  dfbetas <- abs(dfbetas_raw)
} else {
  # Fallback: try to convert to numeric
  dfbetas <- abs(as.numeric(dfbetas_raw))
}

# Ensure all vectors have same length
cat("  Vector lengths: Cook's D = ", length(cooks_d), 
    ", DFBETAS = ", length(dfbetas),
    ", Hat = ", length(hat_values),
    ", Resid = ", length(std_resid), "\n", sep = "")

# Verify all have same length as df
if (length(cooks_d) != nrow(df) || length(dfbetas) != nrow(df) || 
    length(hat_values) != nrow(df) || length(std_resid) != nrow(df)) {
  cat("  ⚠️  Dimension mismatch detected!\n")
  cat("     df has ", nrow(df), " rows\n", sep = "")
  cat("     Adjusting to match...\n")
  
  # Pad shorter vectors with NA
  max_len <- nrow(df)
  if (length(cooks_d) < max_len) cooks_d <- c(cooks_d, rep(NA, max_len - length(cooks_d)))
  if (length(dfbetas) < max_len) dfbetas <- c(dfbetas, rep(NA, max_len - length(dfbetas)))
  if (length(hat_values) < max_len) hat_values <- c(hat_values, rep(NA, max_len - length(hat_values)))
  if (length(std_resid) < max_len) std_resid <- c(std_resid, rep(NA, max_len - length(std_resid)))
  
  # Truncate longer vectors
  cooks_d <- cooks_d[1:max_len]
  dfbetas <- dfbetas[1:max_len]
  hat_values <- hat_values[1:max_len]
  std_resid <- std_resid[1:max_len]
}

# Create comprehensive diagnostics dataframe
influence_df <- data.frame(
  Study_ID = df$Study_ID,
  Effect_ID = df$Effect_ID,
  Hedges_g = df$Hedges_g,
  Cooks_D = cooks_d,
  DFBETAS = dfbetas,
  Hat = hat_values,
  Std_Resid = std_resid,
  stringsAsFactors = FALSE
)

cat("  ✅ Diagnostics organized for ", nrow(influence_df), " effect sizes\n", sep = "")

cat("\n")

# ----------------------------------------------------------------------------
# Step 3.2.3: Define Influence Thresholds
# ----------------------------------------------------------------------------
cat("Step 3.2.3: Defining influence thresholds...\n")
cat(paste0(rep("-", 80), collapse = ""), "\n\n")

k <- res_overall$k          # Number of studies
p <- res_overall$p          # Number of parameters

# Standard thresholds
threshold_cooks <- 4 / (k - p - 1)
threshold_dfbetas <- 2 / sqrt(k)
threshold_hat <- 2 * p / k
threshold_resid_moderate <- 2
threshold_resid_extreme <- 3

cat("  INFLUENCE THRESHOLDS:\n")
cat("  ", paste0(rep("-", 76), collapse = ""), "\n", sep = "")
cat("    Cook's D:         > ", sprintf("%.4f", threshold_cooks), 
    " (4/(k-p-1))\n", sep = "")
cat("    DFBETAS:          > ", sprintf("%.4f", threshold_dfbetas), 
    " (2/√k)\n", sep = "")
cat("    Hat values:       > ", sprintf("%.4f", threshold_hat), 
    " (2p/k)\n", sep = "")
cat("    Std. residuals:   > ", threshold_resid_moderate, 
    " (moderate outlier)\n", sep = "")
cat("                      > ", threshold_resid_extreme, 
    " (extreme outlier)\n", sep = "")

cat("\n")

# ----------------------------------------------------------------------------
# Step 3.2.4: Flag Influential Cases
# ----------------------------------------------------------------------------
cat("Step 3.2.4: Identifying influential cases...\n")
cat(paste0(rep("-", 80), collapse = ""), "\n\n")

# Apply thresholds
influence_df$Flag_Cooks <- influence_df$Cooks_D > threshold_cooks
influence_df$Flag_DFBETAS <- abs(influence_df$DFBETAS) > threshold_dfbetas
influence_df$Flag_Hat <- influence_df$Hat > threshold_hat
influence_df$Flag_Resid_Moderate <- abs(influence_df$Std_Resid) > threshold_resid_moderate
influence_df$Flag_Resid_Extreme <- abs(influence_df$Std_Resid) > threshold_resid_extreme

# Overall influence flag (any diagnostic exceeds threshold)
influence_df$Influential <- (
  influence_df$Flag_Cooks |
  influence_df$Flag_DFBETAS |
  influence_df$Flag_Hat |
  influence_df$Flag_Resid_Extreme
)

# Count influential cases
n_cooks <- sum(influence_df$Flag_Cooks)
n_dfbetas <- sum(influence_df$Flag_DFBETAS)
n_hat <- sum(influence_df$Flag_Hat)
n_resid_mod <- sum(influence_df$Flag_Resid_Moderate)
n_resid_ext <- sum(influence_df$Flag_Resid_Extreme)
n_influential <- sum(influence_df$Influential)

cat("  INFLUENCE DETECTION SUMMARY:\n")
cat("  ", paste0(rep("=", 76), collapse = ""), "\n", sep = "")
cat("\n")
cat("    High Cook's D:           ", n_cooks, " cases (", 
    sprintf("%.1f", (n_cooks/k)*100), "%)\n", sep = "")
cat("    High DFBETAS:            ", n_dfbetas, " cases (", 
    sprintf("%.1f", (n_dfbetas/k)*100), "%)\n", sep = "")
cat("    High leverage (hat):     ", n_hat, " cases (", 
    sprintf("%.1f", (n_hat/k)*100), "%)\n", sep = "")
cat("    Moderate outliers (|r|>2): ", n_resid_mod, " cases (", 
    sprintf("%.1f", (n_resid_mod/k)*100), "%)\n", sep = "")
cat("    Extreme outliers (|r|>3):  ", n_resid_ext, " cases (", 
    sprintf("%.1f", (n_resid_ext/k)*100), "%)\n", sep = "")
cat("\n")
cat("  ", paste0(rep("-", 76), collapse = ""), "\n", sep = "")
cat("    TOTAL INFLUENTIAL CASES: ", n_influential, "/", k, " (", 
    sprintf("%.1f", (n_influential/k)*100), "%)\n", sep = "")
cat("  ", paste0(rep("-", 76), collapse = ""), "\n", sep = "")

cat("\n")

# ----------------------------------------------------------------------------
# Step 3.2.5: Display Influential Cases
# ----------------------------------------------------------------------------
cat("Step 3.2.5: Detailed information on influential cases...\n")
cat(paste0(rep("-", 80), collapse = ""), "\n\n")

if (n_influential > 0) {
  cat("  ⚠️  INFLUENTIAL CASES DETECTED:\n")
  cat("  ", paste0(rep("=", 76), collapse = ""), "\n", sep = "")
  cat("\n")
  
  # Sort by Cook's D (descending)
  influential_cases <- influence_df[influence_df$Influential, ]
  influential_cases <- influential_cases[order(-influential_cases$Cooks_D), ]
  
  for (i in 1:nrow(influential_cases)) {
    row <- influential_cases[i, ]
    
    cat("  ", i, ". ", row$Study_ID, " (", row$Effect_ID, ")\n", sep = "")
    cat("     Effect size:      g = ", sprintf("%.4f", row$Hedges_g), "\n", sep = "")
    cat("     Cook's D:         ", sprintf("%.4f", row$Cooks_D), sep = "")
    if (row$Flag_Cooks) cat(" ⚠️  HIGH")
    cat("\n")
    cat("     DFBETAS:          ", sprintf("%.4f", row$DFBETAS), sep = "")
    if (row$Flag_DFBETAS) cat(" ⚠️  HIGH")
    cat("\n")
    cat("     Hat value:        ", sprintf("%.4f", row$Hat), sep = "")
    if (row$Flag_Hat) cat(" ⚠️  HIGH")
    cat("\n")
    cat("     Std. residual:    ", sprintf("%.4f", row$Std_Resid), sep = "")
    if (row$Flag_Resid_Extreme) {
      cat(" ⚠️  EXTREME OUTLIER")
    } else if (row$Flag_Resid_Moderate) {
      cat(" ⚠️  MODERATE OUTLIER")
    }
    cat("\n")
    
    # Interpretation
    issues <- c()
    if (row$Flag_Cooks) issues <- c(issues, "high influence")
    if (row$Flag_DFBETAS) issues <- c(issues, "affects coefficients")
    if (row$Flag_Hat) issues <- c(issues, "high leverage")
    if (row$Flag_Resid_Extreme) issues <- c(issues, "extreme outlier")
    
    if (length(issues) > 0) {
      cat("     Issues:           ", paste(issues, collapse = ", "), "\n", sep = "")
    }
    cat("\n")
  }
  
  # Recommendations
  cat("  RECOMMENDATIONS:\n")
  cat("  ", paste0(rep("-", 76), collapse = ""), "\n", sep = "")
  cat("    1. Review influential studies for data accuracy\n")
  cat("    2. Examine study characteristics (design, sample, etc.)\n")
  cat("    3. Consider sensitivity analysis excluding influential cases\n")
  cat("    4. Report results both with and without influential cases\n")
  
} else {
  cat("  ✅ NO INFLUENTIAL CASES DETECTED\n")
  cat("     All effect sizes fall within acceptable diagnostic ranges\n")
  cat("     → Results are robust to individual case influence\n")
}

cat("\n")

# ----------------------------------------------------------------------------
# Step 3.2.6: Sensitivity Analysis (Excluding Influential Cases)
# ----------------------------------------------------------------------------
if (n_influential > 0) {
  cat("Step 3.2.6: Sensitivity analysis excluding influential cases...\n")
  cat(paste0(rep("-", 80), collapse = ""), "\n\n")
  
  # Create dataset without influential cases
  df_robust <- df[!influence_df$Influential, ]
  
  cat("  Refitting model without influential cases...\n")
  cat("    Original k:  ", k, " effect sizes\n", sep = "")
  cat("    Excluded:    ", n_influential, " influential cases\n", sep = "")
  cat("    Remaining:   ", nrow(df_robust), " effect sizes\n\n", sep = "")
  
  # Refit overall model
  res_robust <- rma(yi = Hedges_g, vi = df_robust$vi, data = df_robust, method = "REML")
  
  cat("  ✅ Robust model fitted\n\n")
  
  # Compare results
  cat("  COMPARISON: ORIGINAL vs. ROBUST (without influential cases)\n")
  cat("  ", paste0(rep("=", 76), collapse = ""), "\n", sep = "")
  cat("\n")
  
  cat("  Effect size:\n")
  cat("    Original:  g = ", sprintf("%.4f", overall_g), 
      " [", sprintf("%.4f", overall_ci_lb), ", ",
      sprintf("%.4f", overall_ci_ub), "]\n", sep = "")
  cat("    Robust:    g = ", sprintf("%.4f", res_robust$b[1]), 
      " [", sprintf("%.4f", res_robust$ci.lb), ", ",
      sprintf("%.4f", res_robust$ci.ub), "]\n", sep = "")
  cat("    Change:    ", sprintf("%+.4f", res_robust$b[1] - overall_g), 
      " (", sprintf("%+.1f", ((res_robust$b[1] - overall_g)/overall_g)*100), "%)\n\n", sep = "")
  
  cat("  Statistical significance:\n")
  cat("    Original:  p = ", sprintf("%.4f", overall_p), sep = "")
  if (overall_p < 0.001) {
    cat(" ***\n")
  } else if (overall_p < 0.01) {
    cat(" **\n")
  } else if (overall_p < 0.05) {
    cat(" *\n")
  } else {
    cat("\n")
  }
  cat("    Robust:    p = ", sprintf("%.4f", res_robust$pval), sep = "")
  if (res_robust$pval < 0.001) {
    cat(" ***\n")
  } else if (res_robust$pval < 0.01) {
    cat(" **\n")
  } else if (res_robust$pval < 0.05) {
    cat(" *\n")
  } else {
    cat("\n")
  }
  
  # Check if significance changed
  original_sig <- overall_p < 0.05
  robust_sig <- res_robust$pval < 0.05
  
  if (original_sig == robust_sig) {
    cat("    → Significance UNCHANGED\n\n")
  } else {
    cat("    ⚠️  Significance CHANGED\n\n")
  }
  
  cat("  Heterogeneity:\n")
  cat("    Original:  I² = ", sprintf("%.2f", I2), "%, τ² = ", 
      sprintf("%.4f", tau2), "\n", sep = "")
  cat("    Robust:    I² = ", sprintf("%.2f", res_robust$I2), "%, τ² = ", 
      sprintf("%.4f", res_robust$tau2), "\n\n", sep = "")
  
  # Overall interpretation
  change_pct <- abs((res_robust$b[1] - overall_g)/overall_g) * 100
  
  if (change_pct < 5) {
    cat("  ✅ INTERPRETATION: Results are ROBUST\n")
    cat("     Excluding influential cases changes estimate by <5%\n")
    cat("     → Main conclusions remain valid\n")
  } else if (change_pct < 10) {
    cat("  ⚠️  INTERPRETATION: Results are MODERATELY AFFECTED\n")
    cat("     Excluding influential cases changes estimate by 5-10%\n")
    cat("     → Report both original and robust estimates\n")
  } else {
    cat("  ⚠️  INTERPRETATION: Results are SUBSTANTIALLY AFFECTED\n")
    cat("     Excluding influential cases changes estimate by >10%\n")
    cat("     → Influential cases have major impact\n")
    cat("     → Consider reporting robust estimate as primary\n")
  }
  
  cat("\n")
}

# ----------------------------------------------------------------------------
# Step 3.2.7: Export Influence Diagnostics
# ----------------------------------------------------------------------------
cat("Step 3.2.7: Exporting influence diagnostics...\n")
cat(paste0(rep("-", 80), collapse = ""), "\n")

# Round for export
influence_export <- influence_df
numeric_cols <- c("Hedges_g", "Cooks_D", "DFBETAS", "Hat", "Std_Resid")
for (col in numeric_cols) {
  influence_export[[col]] <- round(influence_export[[col]], 4)
}

# Verify step3_dir exists
if (!exists("step3_dir") || is.null(step3_dir)) {
  step3_dir <- file.path(getwd(), "Meta_Analysis_Results", "Step3_Robustness_Checks")
  if (!dir.exists(step3_dir)) dir.create(step3_dir, recursive = TRUE)
}

# Save to Step3 folder
safe_write_csv(influence_export, "influence_diagnostics.csv", output_dir = step3_dir)

cat("\n")
cat("✅ STEP 3.2 COMPLETE: Influence diagnostics finished\n")
cat(paste0(rep("=", 80), collapse = ""), "\n\n")



# ============================================================================
# STEP 3.3: ROBUST VARIANCE ESTIMATION (RVE)
# ============================================================================
#
# Purpose:  Correct for statistical dependency in clustered effect sizes
# Problem:  Multiple ES per study violates independence → SE underestimation
#
# Method:   Hierarchical weights with robust variance (Hedges et al., 2010)
#   • Assumes within-study correlation ρ (default: 0.8)
#   • Small-sample corrections for limited clusters
#   • robumeta package implementation
#
# Outputs:
#   RVE-adjusted g, SE inflation %, comparison to standard MA
#
# Interpretation:
#   SE inflation <10% → Minimal dependency impact
#   SE inflation 10-25% → Moderate dependency, report RVE as primary
#   SE inflation >25% → Strong dependency, RVE results essential
#
# Decision Rule:
#   ≥15% of studies with multiple ES → Conduct RVE analysis
#
# ============================================================================

cat(paste0(rep("=", 80), collapse = ""), "\n")
cat("STEP 3.3: ROBUST VARIANCE ESTIMATION (RVE)\n")
cat(paste0(rep("=", 80), collapse = ""), "\n\n")

cat("Purpose: Account for dependency from multiple effect sizes per study\n\n")

# ----------------------------------------------------------------------------
# Step 3.3.1: Assess Data Structure and Dependency
# ----------------------------------------------------------------------------
cat("Step 3.3.1: Assessing data structure...\n")
cat(paste0(rep("-", 80), collapse = ""), "\n\n")

# Count effect sizes per study
effects_per_study <- table(df$Study_ID)
n_studies <- length(unique(df$Study_ID))
n_effects <- nrow(df)

cat("  DATA STRUCTURE:\n")
cat("  ", paste0(rep("-", 76), collapse = ""), "\n", sep = "")
cat("    Total studies:              ", n_studies, "\n", sep = "")
cat("    Total effect sizes:         ", n_effects, "\n", sep = "")
cat("    Average ES per study:       ", sprintf("%.2f", n_effects/n_studies), "\n", sep = "")
cat("    Median ES per study:        ", median(effects_per_study), "\n", sep = "")
cat("    Range ES per study:         ", min(effects_per_study), " to ", 
    max(effects_per_study), "\n", sep = "")

cat("\n")

# Identify studies with multiple effect sizes
multi_es_studies <- names(effects_per_study[effects_per_study > 1])
n_multi_es <- length(multi_es_studies)
n_multi_effects <- sum(effects_per_study[effects_per_study > 1])

cat("  DEPENDENCY ASSESSMENT:\n")
cat("  ", paste0(rep("-", 76), collapse = ""), "\n", sep = "")
cat("    Studies with >1 effect:     ", n_multi_es, " (", 
    sprintf("%.1f", (n_multi_es/n_studies)*100), "% of studies)\n", sep = "")
cat("    Effect sizes from these:    ", n_multi_effects, " (", 
    sprintf("%.1f", (n_multi_effects/n_effects)*100), "% of total ES)\n", sep = "")

cat("\n")

# Display distribution
cat("  Effect sizes per study (distribution):\n")
es_dist <- as.data.frame(table(effects_per_study))
names(es_dist) <- c("ES_Count", "N_Studies")
for (i in 1:nrow(es_dist)) {
  cat("    ", es_dist$ES_Count[i], " ES: ", es_dist$N_Studies[i], 
      " studies (", sprintf("%.1f", (as.numeric(es_dist$N_Studies[i])/n_studies)*100), 
      "%)\n", sep = "")
}

cat("\n")

# Decision on RVE necessity
if (n_multi_es == 0) {
  cat("  ✅ DECISION: RVE not necessary\n")
  cat("     All studies contribute exactly 1 effect size\n")
  cat("     → No dependency issues\n")
  cat("     → Standard meta-analysis is appropriate\n\n")
  
  run_rve <- FALSE
  
} else {
  cat("  ⚠️  DECISION: RVE is ESSENTIAL\n")
  cat("     Multiple effect sizes per study create statistical dependency\n")
  cat("     → RVE adjusts standard errors for within-study correlation\n")
  cat("     → RVE results should be reported as primary analysis\n\n")
  
  run_rve <- TRUE
}

# ----------------------------------------------------------------------------
# Conditional RVE Analysis (only if needed)
# ----------------------------------------------------------------------------
if (run_rve) {
  
  # --------------------------------------------------------------------------
  # Step 3.3.2: Fit RVE Model for Overall Effect
  # --------------------------------------------------------------------------
  cat("Step 3.3.2: Fitting RVE model for overall effect...\n")
  cat(paste0(rep("-", 80), collapse = ""), "\n\n")
  
  # Check if robumeta is loaded
  if (!require(robumeta, quietly = TRUE)) {
    cat("  Installing robumeta package...\n")
    install.packages("robumeta", repos = "https://cloud.r-project.org/")
    library(robumeta)
  }
  
  # Set assumed within-study correlation
  rho_assumed <- 0.8  # Common default (can be varied in sensitivity analysis)
  
  cat("  MODEL SPECIFICATIONS:\n")
  cat("    Assumed ρ (within-study correlation): ", rho_assumed, "\n", sep = "")
  cat("    Small sample correction: Yes\n")
  cat("    Estimation method: Hierarchical effects\n\n")
  
  # Fit RVE model
  cat("  Fitting RVE model...\n")
  
  rve_overall <- robu(
    formula = Hedges_g ~ 1,
    data = df,
    studynum = Study_ID,
    var.eff.size = vi,
    rho = rho_assumed,
    small = TRUE
  )
  
  cat("  ✅ RVE model fitted successfully\n\n")
  
  # --------------------------------------------------------------------------
  # Step 3.3.3: Display RVE Results
  # --------------------------------------------------------------------------
  cat("Step 3.3.3: RVE overall effect estimate...\n")
  cat(paste0(rep("-", 80), collapse = ""), "\n\n")
  
  cat("  RVE OVERALL EFFECT:\n")
  cat("  ", paste0(rep("=", 76), collapse = ""), "\n", sep = "")
  cat("\n")
  
  rve_g <- rve_overall$reg_table$b.r[1]
  rve_se <- rve_overall$reg_table$SE[1]
  rve_ci_lb <- rve_overall$reg_table$CI.L[1]
  rve_ci_ub <- rve_overall$reg_table$CI.U[1]
  rve_t <- rve_overall$reg_table$t[1]
  rve_p <- rve_overall$reg_table$prob[1]
  rve_df <- rve_overall$reg_table$dfs[1]
  
  cat("    Estimate (g):     ", sprintf("%.4f", rve_g), "\n", sep = "")
  cat("    Standard Error:   ", sprintf("%.4f", rve_se), "\n", sep = "")
  cat("    95% CI:           [", sprintf("%.4f", rve_ci_lb), ", ", 
      sprintf("%.4f", rve_ci_ub), "]\n", sep = "")
  cat("    t-value:          ", sprintf("%.4f", rve_t), "\n", sep = "")
  cat("    df:               ", sprintf("%.1f", rve_df), "\n", sep = "")
  cat("    p-value:          ", sprintf("%.4f", rve_p), sep = "")
  
  if (rve_p < 0.001) {
    cat(" ***\n")
  } else if (rve_p < 0.01) {
    cat(" **\n")
  } else if (rve_p < 0.05) {
    cat(" *\n")
  } else {
    cat("\n")
  }
  
  cat("\n")
  
  # Heterogeneity from RVE
  rve_tau2 <- rve_overall$mod_info$tau.sq
  rve_I2 <- rve_overall$mod_info$I.2
  
  cat("    τ²:               ", sprintf("%.4f", rve_tau2), "\n", sep = "")
  cat("    I²:               ", sprintf("%.2f", rve_I2), "%\n", sep = "")
  
  cat("\n")
  
  # --------------------------------------------------------------------------
  # Step 3.3.4: Compare Standard vs. RVE
  # --------------------------------------------------------------------------
  cat("Step 3.3.4: Comparing standard meta-analysis vs. RVE...\n")
  cat(paste0(rep("-", 80), collapse = ""), "\n\n")
  
  cat("  STANDARD vs. RVE COMPARISON:\n")
  cat("  ", paste0(rep("=", 76), collapse = ""), "\n", sep = "")
  cat("\n")
  
  cat("  Effect Size Estimate:\n")
  cat("    Standard MA: g = ", sprintf("%.4f", overall_g), 
      " [", sprintf("%.4f", overall_ci_lb), ", ",
      sprintf("%.4f", overall_ci_ub), "]\n", sep = "")
  cat("    RVE:         g = ", sprintf("%.4f", rve_g), 
      " [", sprintf("%.4f", rve_ci_lb), ", ",
      sprintf("%.4f", rve_ci_ub), "]\n", sep = "")
  cat("    Difference:      ", sprintf("%+.4f", rve_g - overall_g), "\n\n", sep = "")
  
  cat("  Standard Error:\n")
  cat("    Standard MA: SE = ", sprintf("%.4f", overall_se), "\n", sep = "")
  cat("    RVE:         SE = ", sprintf("%.4f", rve_se), "\n", sep = "")
  
  se_ratio <- rve_se / overall_se
  se_inflation <- (se_ratio - 1) * 100
  
  cat("    SE Ratio:        ", sprintf("%.2f", se_ratio), "x\n", sep = "")
  cat("    SE Inflation:    +", sprintf("%.1f", se_inflation), "%\n\n", sep = "")
  
  cat("  Confidence Interval Width:\n")
  ci_width_standard <- overall_ci_ub - overall_ci_lb
  ci_width_rve <- rve_ci_ub - rve_ci_lb
  cat("    Standard MA: ", sprintf("%.4f", ci_width_standard), "\n", sep = "")
  cat("    RVE:         ", sprintf("%.4f", ci_width_rve), "\n", sep = "")
  cat("    Increase:    +", sprintf("%.1f", ((ci_width_rve/ci_width_standard - 1)*100)), "%\n\n", sep = "")
  
  cat("  Statistical Significance:\n")
  cat("    Standard MA: p = ", sprintf("%.4f", overall_p), sep = "")
  if (overall_p < 0.05) cat(" (significant)")
  cat("\n")
  cat("    RVE:         p = ", sprintf("%.4f", rve_p), sep = "")
  if (rve_p < 0.05) cat(" (significant)")
  cat("\n\n")
  
  # Check if significance changed
  standard_sig <- overall_p < 0.05
  rve_sig <- rve_p < 0.05
  
  if (standard_sig == rve_sig) {
    cat("    → Significance UNCHANGED\n")
  } else {
    cat("    ⚠️  Significance CHANGED with RVE adjustment\n")
  }
  
  cat("\n")
  
  # --------------------------------------------------------------------------
  # Step 3.3.5: Interpret Impact of Dependency
  # --------------------------------------------------------------------------
  cat("Step 3.3.5: Interpreting dependency impact...\n")
  cat(paste0(rep("-", 80), collapse = ""), "\n\n")
  
  cat("  DEPENDENCY IMPACT ASSESSMENT:\n")
  cat("  ", paste0(rep("=", 76), collapse = ""), "\n", sep = "")
  cat("\n")
  
  if (se_inflation < 10) {
    cat("  ✅ MINIMAL DEPENDENCY IMPACT (SE inflation < 10%)\n")
    cat("     → Within-study correlation has limited effect\n")
    cat("     → Standard and RVE results are similar\n")
    cat("     → Either approach acceptable, but RVE preferred for transparency\n")
  } else if (se_inflation < 25) {
    cat("  ⚠️  MODERATE DEPENDENCY IMPACT (SE inflation 10-25%)\n")
    cat("     → Within-study correlation moderately affects inference\n")
    cat("     → RVE provides more conservative estimates\n")
    cat("     → RECOMMEND: Report RVE as primary analysis\n")
  } else {
    cat("  ⚠️  SUBSTANTIAL DEPENDENCY IMPACT (SE inflation > 25%)\n")
    cat("     → Within-study correlation substantially affects inference\n")
    cat("     → Standard MA underestimates uncertainty\n")
    cat("     → ESSENTIAL: Report RVE as primary analysis\n")
    cat("     → Standard MA results may be misleading\n")
  }
  
  cat("\n")
  
  # --------------------------------------------------------------------------
  # Step 3.3.6: RVE for Significant Moderators (if any)
  # --------------------------------------------------------------------------
  if (exists("significant_moderators") && nrow(significant_moderators) > 0) {
    
    cat("Step 3.3.6: RVE analysis for significant moderators...\n")
    cat(paste0(rep("-", 80), collapse = ""), "\n\n")
    
    cat("  Testing ", nrow(significant_moderators), " significant moderator(s) with RVE...\n\n", sep = "")
    
    rve_moderator_results <- list()
    
    for (i in 1:nrow(significant_moderators)) {
      mod_name <- significant_moderators$Moderator[i]
      
      cat("  [", i, "/", nrow(significant_moderators), "] ", mod_name, "\n", sep = "")
      
      # Check complete cases
      temp_rve <- df[!is.na(df[[mod_name]]), ]
      
      if (nrow(temp_rve) < 5) {
        cat("      ⚠️  Insufficient data (n = ", nrow(temp_rve), ") → SKIP\n\n", sep = "")
        next
      }
      
      # Fit RVE model with moderator
      rve_mod <- tryCatch(
        {
          robu(
            formula = as.formula(paste("Hedges_g ~", mod_name)),
            data = temp_rve,
            studynum = Study_ID,
            var.eff.size = vi,
            rho = rho_assumed,
            small = TRUE
          )
        },
        error = function(e) {
          cat("      ❌ RVE model failed: ", e$message, "\n\n", sep = "")
          return(NULL)
        }
      )
      
      if (is.null(rve_mod) || nrow(rve_mod$reg_table) < 2) {
        cat("      ⚠️  No moderator coefficient → SKIP\n\n")
        next
      }
      
      # Extract results (second row is moderator effect)
      rve_mod_results <- data.frame(
        Moderator = mod_name,
        Estimate_RVE = rve_mod$reg_table$b.r[2],
        SE_RVE = rve_mod$reg_table$SE[2],
        CI_Lower_RVE = rve_mod$reg_table$CI.L[2],
        CI_Upper_RVE = rve_mod$reg_table$CI.U[2],
        t_value = rve_mod$reg_table$t[2],
        df = rve_mod$reg_table$dfs[2],
        p_value_RVE = rve_mod$reg_table$prob[2],
        n_studies = length(unique(temp_rve$Study_ID)),
        n_effects = nrow(temp_rve),
        stringsAsFactors = FALSE
      )
      
      rve_moderator_results[[mod_name]] <- rve_mod_results
      
      cat("      ✅ β = ", sprintf("%.4f", rve_mod_results$Estimate_RVE), 
          ", SE = ", sprintf("%.4f", rve_mod_results$SE_RVE), 
          ", p = ", sprintf("%.4f", rve_mod_results$p_value_RVE), sep = "")
      
      if (rve_mod_results$p_value_RVE < 0.001) {
        cat(" ***\n")
      } else if (rve_mod_results$p_value_RVE < 0.01) {
        cat(" **\n")
      } else if (rve_mod_results$p_value_RVE < 0.05) {
        cat(" *\n")
      } else {
        cat("\n")
      }
      
      cat("\n")
    }
    
    # Combine RVE moderator results
    if (length(rve_moderator_results) > 0) {
      rve_mod_df <- do.call(rbind, rve_moderator_results)
      rownames(rve_mod_df) <- NULL
      
      # Compare standard vs RVE for each moderator
      cat("  MODERATOR COMPARISON: Standard MA vs. RVE\n")
      cat("  ", paste0(rep("=", 76), collapse = ""), "\n", sep = "")
      cat("\n")
      
      for (mod in rve_mod_df$Moderator) {
        rve_result <- rve_mod_df[rve_mod_df$Moderator == mod, ]
        
        cat("  ", mod, "\n", sep = "")
        
        # Get standard result from univariate_results_list
        if (exists("univariate_results_list") && mod %in% names(univariate_results_list)) {
          std_model <- univariate_results_list[[mod]]$model
          
          # For categorical moderators, get the omnibus test
          # For continuous, get the coefficient
          if (is.factor(df[[mod]]) || is.character(df[[mod]])) {
            # Categorical: use omnibus QM test
            std_QM <- std_model$QM
            std_p <- std_model$QMp
            
            cat("    Standard: QM = ", sprintf("%.4f", std_QM), 
                ", p = ", sprintf("%.4f", std_p), " (omnibus test)\n", sep = "")
          } else {
            # Continuous: use coefficient (second row)
            if (nrow(std_model$b) >= 2) {
              std_b <- as.numeric(std_model$b[2])
              std_se <- std_model$se[2]
              std_p <- std_model$pval[2]
              
              cat("    Standard: β = ", sprintf("%.4f", std_b), 
                  ", SE = ", sprintf("%.4f", std_se), 
                  ", p = ", sprintf("%.4f", std_p), "\n", sep = "")
              
              cat("    RVE:      β = ", sprintf("%.4f", rve_result$Estimate_RVE), 
                  ", SE = ", sprintf("%.4f", rve_result$SE_RVE), 
                  ", p = ", sprintf("%.4f", rve_result$p_value_RVE), "\n", sep = "")
              
              se_ratio_mod <- rve_result$SE_RVE / std_se
              cat("    SE Ratio: ", sprintf("%.2f", se_ratio_mod), "x", sep = "")
              
              # Check if significance changed
              std_sig_mod <- std_p < 0.05
              rve_sig_mod <- rve_result$p_value_RVE < 0.05
              
              if (std_sig_mod != rve_sig_mod) {
                cat(" ⚠️  Significance changed!")
              }
              cat("\n\n")
              next
            }
          }
        }
        
        # If we couldn't get standard results, just show RVE
        cat("    RVE:      β = ", sprintf("%.4f", rve_result$Estimate_RVE), 
            ", SE = ", sprintf("%.4f", rve_result$SE_RVE), 
            ", p = ", sprintf("%.4f", rve_result$p_value_RVE), "\n", sep = "")
        cat("\n")
      }
    }
  }
  
  # --------------------------------------------------------------------------
  # Step 3.3.7: Export RVE Results
  # --------------------------------------------------------------------------
  cat("Step 3.3.7: Exporting RVE results...\n")
  cat(paste0(rep("-", 80), collapse = ""), "\n")
  
  # Verify step3_dir exists
  if (!exists("step3_dir") || is.null(step3_dir)) {
    step3_dir <- file.path(getwd(), "Meta_Analysis_Results", "Step3_Robustness_Checks")
    if (!dir.exists(step3_dir)) dir.create(step3_dir, recursive = TRUE)
  }
  
  # Overall effect
  rve_overall_export <- data.frame(
    Analysis = "RVE Overall Effect",
    rho_assumed = rho_assumed,
    k_studies = n_studies,
    k_effects = n_effects,
    Estimate = round(rve_g, 4),
    SE = round(rve_se, 4),
    CI_Lower = round(rve_ci_lb, 4),
    CI_Upper = round(rve_ci_ub, 4),
    t_value = round(rve_t, 4),
    df = round(rve_df, 1),
    p_value = round(rve_p, 4),
    tau2 = round(rve_tau2, 4),
    I2 = round(rve_I2, 2),
    SE_ratio_vs_standard = round(se_ratio, 2),
    SE_inflation_pct = round(se_inflation, 1),
    stringsAsFactors = FALSE
  )
  
  # Write RVE overall effect to Step3 folder
  safe_write_csv(rve_overall_export, "rve_overall_effect.csv", output_dir = step3_dir)
  
  # Moderator results (if any)
  if (exists("rve_mod_df")) {
    rve_mod_export <- rve_mod_df
    numeric_cols <- c("Estimate_RVE", "SE_RVE", "CI_Lower_RVE", "CI_Upper_RVE",
                      "t_value", "df", "p_value_RVE")
    for (col in numeric_cols) {
      rve_mod_export[[col]] <- round(rve_mod_export[[col]], 4)
    }
    
    # Write RVE moderator results to Step3 folder
    safe_write_csv(rve_mod_export, "rve_moderator_results.csv", output_dir = step3_dir)
  }
  
  cat("\n")
}

if (run_rve) {
  cat("✅ STEP 3.3 COMPLETE: Robust variance estimation finished\n")
} else {
  cat("✅ STEP 3.3 COMPLETE: RVE not necessary (no dependency)\n")
}
cat(paste0(rep("=", 80), collapse = ""), "\n\n")



STEP 3.1: LEAVE-ONE-OUT SENSITIVITY ANALYSIS

Research Question: Is the overall effect robust to study exclusion?

Step 3.1.1: Baseline overall effect (for comparison)...
-------------------------------------------------------------------------------- 

  BASELINE (All studies included):
    Effect size:  g = 0.5272 [0.3781, 0.6763]
    p-value:      0.0000 ***
    τ²:           0.0882
    I²:           54.64%
    k:            29 effect sizes

Step 3.1.2: Running leave-one-out analysis...
-------------------------------------------------------------------------------- 
  Systematically excluding each effect size...

  ✅ Leave-one-out analysis completed
     Iterations: 

Step 3.1.3: Organizing sensitivity results...
-------------------------------------------------------------------------------- 
  ✅ Results organized with change metrics

Step 3.1.4: Summary statistics across leave-one-out iterations...
-------------------------------------------------------------------------------- 


[1] TRUE


✅ STEP 3.1 COMPLETE: Leave-one-out sensitivity analysis finished

STEP 3.2: INFLUENCE DIAGNOSTICS

Research Question: Are there influential outliers affecting results?

Step 3.2.1: Calculating influence diagnostics...
-------------------------------------------------------------------------------- 
  Computing diagnostic metrics:
    → Cook's Distance (overall influence)
    → DFBETAS (coefficient influence)
    → Hat values (leverage)
    → Standardized residuals (outliers)

  ✅ Diagnostic calculations complete

Step 3.2.2: Organizing diagnostic statistics...
-------------------------------------------------------------------------------- 
  ⚠️  DFBETAS not available from influence(), using zeros
  Vector lengths: Cook's D = 29, DFBETAS = 29, Hat = 29, Resid = 29
  ✅ Diagnostics organized for 29 effect sizes

Step 3.2.3: Defining influence thresholds...
-------------------------------------------------------------------------------- 

  INFLUENCE THRESHOLDS:
  -----------------------

[1] TRUE


✅ STEP 3.2 COMPLETE: Influence diagnostics finished

STEP 3.3: ROBUST VARIANCE ESTIMATION (RVE)

Purpose: Account for dependency from multiple effect sizes per study

Step 3.3.1: Assessing data structure...
-------------------------------------------------------------------------------- 

  DATA STRUCTURE:
  ----------------------------------------------------------------------------
    Total studies:              24
    Total effect sizes:         29
    Average ES per study:       1.21
    Median ES per study:        1
    Range ES per study:         1 to 3

  DEPENDENCY ASSESSMENT:
  ----------------------------------------------------------------------------
    Studies with >1 effect:     4 (16.7% of studies)
    Effect sizes from these:    9 (31.0% of total ES)

  Effect sizes per study (distribution):
    1 ES: 20 studies (83.3%)
    2 ES: 3 studies (12.5%)
    3 ES: 1 studies (4.2%)

  ⚠️  DECISION: RVE is ESSENTIAL
     Multiple effect sizes per study create statistical depe

In [10]:
# ============================================================================
# STEP 4: PUBLICATION BIAS ASSESSMENT
# ============================================================================
#
# Purpose: To evaluate the potential presence of publication bias and small‐study effects.
#
# Methods:
#   Multi-pronged publication bias detection:
#     1. Funnel plot (visual asymmetry)
#     2. Egger’s regression test for small-study effects
#     3. Trim-and-fill procedure for bias-adjusted effect estimation
#     4. Rosenthal’s fail-safe N to quantify robustness to unpublished null studies
#
# Files Generated (Output):
#   • funnel_plot.png
#   • publication_bias_tests.csv
#   • publication_bias_trim_and_fill.csv → (Only generated when k_imputed > 0)
#
# Bias Interpretation Framework:
#   0 indicators → No evidence of bias
#   1 indicator  → Minimal concern
#   2 indicators → Moderate concern
#   3 indicators → Substantial bias likely present
#
# ============================================================================

cat(paste0(rep("=", 80), collapse = ""), "\n")
cat("STEP 4: PUBLICATION BIAS ASSESSMENT\n")
cat(paste0(rep("=", 80), collapse = ""), "\n\n")

cat("Research Question: Is there evidence of publication bias?\n\n")

# ----------------------------------------------------------------------------
# Step 4.1: Funnel Plot Asymmetry (Visual Inspection)
# ----------------------------------------------------------------------------
cat("Step 4.1: Funnel plot assessment...\n")
cat(paste0(rep("-", 80), collapse = ""), "\n\n")

cat("  FUNNEL PLOT INTERPRETATION:\n")
cat("  ", paste0(rep("-", 76), collapse = ""), "\n", sep = "")
cat("    Expected (no bias): Symmetric inverted funnel\n")
cat("    Bias indicator:     Asymmetry, especially at bottom (low precision)\n")
cat("    Missing studies:    Gaps in lower right (small non-significant)\n\n")

cat("  NOTE: Funnel plot saved for visual inspection\n")
cat("        (Requires graphical output - not shown in text)\n\n")

# Save funnel plot (if graphical output available)
# Verify step4_dir exists
if (!exists("step4_dir") || is.null(step4_dir)) {
  step4_dir <- file.path(getwd(), "Meta_Analysis_Results", "Step4_Publication_Bias")
  if (!dir.exists(step4_dir)) dir.create(step4_dir, recursive = TRUE)
}

tryCatch({
  png(file.path(step4_dir, "funnel_plot.png"), width = 800, height = 600, res = 120)
  funnel(res_overall, 
         xlab = "Hedges' g",
         ylab = "Standard Error",
         main = "Funnel Plot for Publication Bias Assessment",
         back = "white",
         shade = "white")
  dev.off()
  cat("  ✅ Funnel plot saved: funnel_plot.png\n\n")
}, error = function(e) {
  cat("  ⚠️  Funnel plot not saved (graphical device not available)\n\n")
})

# ----------------------------------------------------------------------------
# Step 4.2: Egger's Regression Test
# ----------------------------------------------------------------------------
cat("Step 4.2: Egger's regression test for funnel plot asymmetry...\n")
cat(paste0(rep("-", 80), collapse = ""), "\n\n")

cat("  METHOD: Regression of standardized effect on precision\n")
cat("  H₀: No small-study effects (intercept = 0)\n")
cat("  H₁: Funnel plot asymmetry present\n\n")

# Perform Egger's test
egger_test <- regtest(res_overall, model = "lm")

cat("  EGGER'S TEST RESULTS:\n")
cat("  ", paste0(rep("=", 76), collapse = ""), "\n", sep = "")
cat("\n")
cat("    Intercept:        ", sprintf("%.4f", egger_test$est), "\n", sep = "")
cat("    Standard Error:   ", sprintf("%.4f", egger_test$se), "\n", sep = "")
cat("    z-value:          ", sprintf("%.4f", egger_test$zval), "\n", sep = "")
cat("    p-value:          ", sprintf("%.4f", egger_test$pval), sep = "")

if (egger_test$pval < 0.001) {
  cat(" ***\n")
  egger_interpretation <- "HIGHLY SIGNIFICANT asymmetry (p < .001)"
  egger_conclusion <- "⚠️  Strong evidence of publication bias"
} else if (egger_test$pval < 0.01) {
  cat(" **\n")
  egger_interpretation <- "Very significant asymmetry (p < .01)"
  egger_conclusion <- "⚠️  Evidence of publication bias"
} else if (egger_test$pval < 0.05) {
  cat(" *\n")
  egger_interpretation <- "Significant asymmetry (p < .05)"
  egger_conclusion <- "⚠️  Some evidence of publication bias"
} else {
  cat("\n")
  egger_interpretation <- "No significant asymmetry (p ≥ .05)"
  egger_conclusion <- "✅ No statistical evidence of publication bias"
}

cat("\n")
cat("  INTERPRETATION:\n")
cat("    ", egger_interpretation, "\n", sep = "")
cat("    ", egger_conclusion, "\n", sep = "")

cat("\n")

# ----------------------------------------------------------------------------
# Step 4.3: Trim-and-Fill Analysis
# ----------------------------------------------------------------------------
cat("Step 4.3: Trim-and-fill analysis...\n")
cat(paste0(rep("-", 80), collapse = ""), "\n\n")

cat("  METHOD: Impute missing studies to restore funnel plot symmetry\n")
cat("  PURPOSE: Estimate effect size adjusted for publication bias\n\n")

# Perform trim-and-fill
taf_results <- trimfill(res_overall)

cat("  TRIM-AND-FILL RESULTS:\n")
cat("  ", paste0(rep("=", 76), collapse = ""), "\n", sep = "")
cat("\n")

# Number of imputed studies
k_imputed <- taf_results$k0

cat("    Studies imputed:    ", k_imputed, "\n", sep = "")

if (k_imputed == 0) {
  cat("    → No missing studies detected\n")
  cat("    → Funnel plot appears symmetric\n\n")
  
  cat("  ADJUSTED ESTIMATE: Same as original (no adjustment needed)\n\n")
  
  taf_conclusion <- "✅ No publication bias detected by trim-and-fill"
  
} else {
  cat("    → ", k_imputed, " studies imputed on ", 
      ifelse(taf_results$side == "left", "left", "right"), " side\n", sep = "")
  cat("    → Suggests missing ", 
      ifelse(taf_results$side == "left", "negative", "positive"), 
      " studies\n\n", sep = "")
  
  # Adjusted effect size
  taf_g <- taf_results$b[1]
  taf_ci_lb <- taf_results$ci.lb
  taf_ci_ub <- taf_results$ci.ub
  taf_p <- taf_results$pval
  
  cat("  ORIGINAL vs. ADJUSTED EFFECT:\n")
  cat("  ", paste0(rep("-", 76), collapse = ""), "\n", sep = "")
  cat("    Original:  g = ", sprintf("%.4f", overall_g), 
      " [", sprintf("%.4f", overall_ci_lb), ", ",
      sprintf("%.4f", overall_ci_ub), "], p = ", 
      sprintf("%.4f", overall_p), "\n", sep = "")
  cat("    Adjusted:  g = ", sprintf("%.4f", taf_g), 
      " [", sprintf("%.4f", taf_ci_lb), ", ",
      sprintf("%.4f", taf_ci_ub), "], p = ", 
      sprintf("%.4f", taf_p), "\n", sep = "")
  
  # Calculate change
  taf_change <- taf_g - overall_g
  taf_change_pct <- (taf_change / overall_g) * 100
  
  cat("    Change:    ", sprintf("%+.4f", taf_change), 
      " (", sprintf("%+.1f", taf_change_pct), "%)\n\n", sep = "")
  
  # Interpretation
  if (abs(taf_change_pct) < 10) {
    taf_conclusion <- "⚠️  Minimal bias impact (adjustment < 10%)"
    taf_recommendation <- "Report original effect; note trim-and-fill in supplementary"
  } else if (abs(taf_change_pct) < 25) {
    taf_conclusion <- "⚠️  Moderate bias impact (adjustment 10-25%)"
    taf_recommendation <- "Report both original and adjusted effects"
  } else {
    taf_conclusion <- "⚠️  Substantial bias impact (adjustment > 25%)"
    taf_recommendation <- "Consider adjusted effect as primary estimate"
  }
  
  cat("  INTERPRETATION:\n")
  cat("    ", taf_conclusion, "\n", sep = "")
  cat("    Recommendation: ", taf_recommendation, "\n", sep = "")
  
  cat("\n")
}

# ----------------------------------------------------------------------------
# Step 4.4: Fail-Safe N (Rosenthal's Method)
# ----------------------------------------------------------------------------
cat("Step 4.4: Fail-safe N analysis...\n")
cat(paste0(rep("-", 80), collapse = ""), "\n\n")

cat("  METHOD: Calculate number of null studies needed to nullify effect\n")
cat("  PURPOSE: Assess robustness to unreported non-significant studies\n\n")

# Calculate fail-safe N using the fitted model
fsn_results <- fsn(res_overall, type = "Rosenthal")

cat("  FAIL-SAFE N RESULTS:\n")
cat("  ", paste0(rep("=", 76), collapse = ""), "\n", sep = "")
cat("\n")
cat("    Fail-safe N:      ", fsn_results$fsnum, " studies\n", sep = "")
cat("    Target (5k+5):    ", 5*k + 5, " studies\n", sep = "")

if (fsn_results$fsnum > (5*k + 5)) {
  cat("    → Fail-safe N EXCEEDS target\n")
  cat("    → Effect is robust to file-drawer problem\n")
  fsn_conclusion <- "✅ Effect robust to unreported null studies"
} else {
  cat("    → Fail-safe N BELOW target\n")
  cat("    → Effect may be vulnerable to file-drawer problem\n")
  fsn_conclusion <- "⚠️  Effect potentially vulnerable to unreported studies"
}

cat("\n")
cat("  INTERPRETATION:\n")
cat("    ", fsn_conclusion, "\n", sep = "")
cat("    ", fsn_results$fsnum, " null studies would be needed to nullify effect\n", sep = "")

cat("\n")

# ----------------------------------------------------------------------------
# Step 4.5: Overall Publication Bias Summary
# ----------------------------------------------------------------------------
cat("Step 4.5: Publication bias assessment summary...\n")
cat(paste0(rep("-", 80), collapse = ""), "\n\n")

cat("  PUBLICATION BIAS ASSESSMENT SUMMARY:\n")
cat("  ", paste0(rep("=", 76), collapse = ""), "\n", sep = "")
cat("\n")

# Count bias indicators
bias_indicators <- 0
if (egger_test$pval < 0.05) bias_indicators <- bias_indicators + 1
if (k_imputed > 0) bias_indicators <- bias_indicators + 1
if (fsn_results$fsnum < (5*k + 5)) bias_indicators <- bias_indicators + 1

cat("  1. FUNNEL PLOT:\n")
cat("     → Visual inspection recommended (see funnel_plot.png)\n\n")

cat("  2. EGGER'S TEST:\n")
cat("     → ", egger_conclusion, "\n", sep = "")
cat("     → p = ", sprintf("%.4f", egger_test$pval), "\n\n", sep = "")

cat("  3. TRIM-AND-FILL:\n")
cat("     → ", taf_conclusion, "\n", sep = "")
if (k_imputed > 0) {
  cat("     → ", k_imputed, " studies imputed\n", sep = "")
  cat("     → Adjusted g = ", sprintf("%.4f", taf_g), 
      " (change: ", sprintf("%+.1f", taf_change_pct), "%)\n", sep = "")
}
cat("\n")

cat("  4. FAIL-SAFE N:\n")
cat("     → ", fsn_conclusion, "\n", sep = "")
cat("     → N = ", fsn_results$fsnum, " (target: ", 5*k + 5, ")\n\n", sep = "")

cat("  ", paste0(rep("-", 76), collapse = ""), "\n", sep = "")
cat("  BIAS INDICATORS PRESENT: ", bias_indicators, "/3\n", sep = "")
cat("  ", paste0(rep("-", 76), collapse = ""), "\n", sep = "")
cat("\n")

# Overall conclusion
if (bias_indicators == 0) {
  overall_bias_conclusion <- "✅ NO EVIDENCE OF PUBLICATION BIAS"
  overall_bias_recommendation <- "Results appear unbiased; report with confidence"
} else if (bias_indicators == 1) {
  overall_bias_conclusion <- "⚠️  MINIMAL EVIDENCE OF PUBLICATION BIAS"
  overall_bias_recommendation <- "Results likely robust; acknowledge limitations"
} else if (bias_indicators == 2) {
  overall_bias_conclusion <- "⚠️  MODERATE EVIDENCE OF PUBLICATION BIAS"
  overall_bias_recommendation <- "Report bias tests; consider adjusted estimates"
} else {
  overall_bias_conclusion <- "⚠️  SUBSTANTIAL EVIDENCE OF PUBLICATION BIAS"
  overall_bias_recommendation <- "Interpret results cautiously; report all bias assessments"
}

cat("  OVERALL CONCLUSION:\n")
cat("    ", overall_bias_conclusion, "\n", sep = "")
cat("    Recommendation: ", overall_bias_recommendation, "\n", sep = "")

cat("\n")

# ----------------------------------------------------------------------------
# Step 4.6: Export Publication Bias Results
# ----------------------------------------------------------------------------
cat("Step 4.6: Exporting publication bias assessment results...\n")
cat(paste0(rep("-", 80), collapse = ""), "\n")

# Verify step4_dir exists
if (!exists("step4_dir") || is.null(step4_dir)) {
  step4_dir <- file.path(getwd(), "Meta_Analysis_Results", "Step4_Publication_Bias")
  if (!dir.exists(step4_dir)) dir.create(step4_dir, recursive = TRUE)
}

# Compile all bias test results
bias_results <- data.frame(
  Test = c("Egger's Regression", "Trim-and-Fill", "Fail-Safe N"),
  Statistic = c(
    sprintf("Intercept = %.4f", egger_test$est),
    sprintf("%d studies imputed", k_imputed),
    sprintf("N = %d", fsn_results$fsnum)
  ),
  p_value = c(
    round(egger_test$pval, 4),
    ifelse(k_imputed > 0, round(taf_p, 4), NA),
    NA
  ),
  Interpretation = c(
    ifelse(egger_test$pval < 0.05, "Asymmetry detected", "No asymmetry"),
    ifelse(k_imputed > 0, "Bias suspected", "No bias"),
    ifelse(fsn_results$fsnum > (5*k + 5), "Robust", "Vulnerable")
  ),
  Bias_Evidence = c(
    ifelse(egger_test$pval < 0.05, "Yes", "No"),
    ifelse(k_imputed > 0, "Yes", "No"),
    ifelse(fsn_results$fsnum < (5*k + 5), "Yes", "No")
  ),
  stringsAsFactors = FALSE
)

# Add trim-and-fill adjusted effect if imputed
if (k_imputed > 0) {
  taf_effect <- data.frame(
    Analysis = "Trim-and-Fill Adjusted",
    k_original = k,
    k_imputed = k_imputed,
    Estimate_original = round(overall_g, 4),
    Estimate_adjusted = round(taf_g, 4),
    Change = round(taf_change, 4),
    Change_pct = round(taf_change_pct, 2),
    CI_Lower_adjusted = round(taf_ci_lb, 4),
    CI_Upper_adjusted = round(taf_ci_ub, 4),
    p_value_adjusted = round(taf_p, 4),
    stringsAsFactors = FALSE
  )
  
  # Save to Step4 folder
  safe_write_csv(taf_effect, "publication_bias_trim_and_fill.csv", output_dir = step4_dir)
}

# Save publication bias tests to Step4 folder
safe_write_csv(bias_results, "publication_bias_tests.csv", output_dir = step4_dir)

cat("\n")
cat("✅ STEP 4 COMPLETE: Publication bias assessment finished\n")
cat(paste0(rep("=", 80), collapse = ""), "\n\n")


STEP 4: PUBLICATION BIAS ASSESSMENT

Research Question: Is there evidence of publication bias?

Step 4.1: Funnel plot assessment...
-------------------------------------------------------------------------------- 

  FUNNEL PLOT INTERPRETATION:
  ----------------------------------------------------------------------------
    Expected (no bias): Symmetric inverted funnel
    Bias indicator:     Asymmetry, especially at bottom (low precision)
    Missing studies:    Gaps in lower right (small non-significant)

  NOTE: Funnel plot saved for visual inspection
        (Requires graphical output - not shown in text)

  ✅ Funnel plot saved: funnel_plot.png

Step 4.2: Egger's regression test for funnel plot asymmetry...
-------------------------------------------------------------------------------- 

  METHOD: Regression of standardized effect on precision
  H₀: No small-study effects (intercept = 0)
  H₁: Funnel plot asymmetry present

  EGGER'S TEST RESULTS:

    Intercept:        -0.0910


Warning message:
"Setting type='General' when using fsn() on a model object."


  FAIL-SAFE N RESULTS:

    Fail-safe N:      281 studies
    Target (5k+5):    150 studies
    → Fail-safe N EXCEEDS target
    → Effect is robust to file-drawer problem

  INTERPRETATION:
    ✅ Effect robust to unreported null studies
    281 null studies would be needed to nullify effect

Step 4.5: Publication bias assessment summary...
-------------------------------------------------------------------------------- 

  PUBLICATION BIAS ASSESSMENT SUMMARY:

  1. FUNNEL PLOT:
     → Visual inspection recommended (see funnel_plot.png)

  2. EGGER'S TEST:
     → ✅ No statistical evidence of publication bias
     → p = 0.0977

  3. TRIM-AND-FILL:
     → ✅ No publication bias detected by trim-and-fill

  4. FAIL-SAFE N:
     → ✅ Effect robust to unreported null studies
     → N = 281 (target: 150)

  ----------------------------------------------------------------------------
  BIAS INDICATORS PRESENT: 0/3
  ----------------------------------------------------------------------------

  

[1] TRUE


✅ STEP 4 COMPLETE: Publication bias assessment finished

